### 불러오기

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
from sklearn.preprocessing import StandardScaler,MinMaxScaler
from sklearn.model_selection import GridSearchCV,train_test_split
from imblearn.over_sampling import SMOTE

In [3]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier,GradientBoostingClassifier
from sklearn.metrics import classification_report, accuracy_score, f1_score
from sklearn.svm import SVC

In [4]:
from sklearn.feature_selection import SelectFromModel

In [5]:
from xgboost import XGBClassifier

In [6]:
np.set_printoptions(suppress=True)
np.set_printoptions(threshold=np.inf)

In [149]:
base1 = pd.read_csv('./data/BASE_DATA1.csv',encoding='cp949')
base2 = pd.read_csv('./data/BASE_DATA2.csv',encoding='cp949')
base3 = pd.read_csv('./data/BASE_DATA3.csv',encoding='cp949')

In [150]:
print(base1.info())
print(base2.info())
print(base3.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   T_ID          10000 non-null  object
 1   T_DATA_CLASS  10000 non-null  object
 2   T_EDATE       10000 non-null  int64 
 3   T_SEX         10000 non-null  int64 
 4   T_AGE         10000 non-null  int64 
 5   T_INCOME      10000 non-null  int64 
 6   T_MARRY       10000 non-null  int64 
dtypes: int64(5), object(2)
memory usage: 547.0+ KB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 39 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   T_ID          10000 non-null  object 
 1   T_DATA_CLASS  10000 non-null  object 
 2   T_HTN         10000 non-null  int64  
 3   T_HTNAG       10000 non-null  int64  
 4   T_DM          10000 non-null  int64  
 5   T_DMAG        10000 non-null  int64  
 6   T

### 변수 선택

In [156]:
base_merged = pd.merge(pd.merge(base1,base2,on=['T_ID','T_DATA_CLASS']),base3,on=['T_ID','T_DATA_CLASS'])

In [157]:
base_merged.columns

Index(['T_ID', 'T_DATA_CLASS', 'T_EDATE', 'T_SEX', 'T_AGE', 'T_INCOME',
       'T_MARRY', 'T_HTN', 'T_HTNAG', 'T_DM', 'T_DMAG', 'T_LIP', 'T_LIPAG',
       'T_FMFHT1', 'T_FMFHT2', 'T_FMFDM1', 'T_FMFDM2', 'T_DRINK', 'T_DRDU',
       'T_TAKFQ', 'T_TAKAM', 'T_RICEFQ', 'T_RICEAM', 'T_WINEFQ', 'T_WINEAM',
       'T_SOJUFQ', 'T_SOJUAM', 'T_BEERFQ', 'T_BEERAM', 'T_HLIQFQ', 'T_HLIQAM',
       'T_TOTALC', 'T_SMOKE', 'T_SMDUYR', 'T_SMDUMO', 'T_SMAM', 'T_PACKYR',
       'T_PSM', 'T_EXER', 'T_MNSAG', 'T_PMYN', 'T_PMAG', 'T_PREG', 'T_FPREGAG',
       'T_SBP', 'T_DBP', 'T_PULSE', 'T_WAIST', 'T_HIP', 'T_HEIGHT', 'T_WEIGHT',
       'T_BMI', 'T_HBA1C', 'T_GLU0', 'T_CREATINE', 'T_AST', 'T_ALT', 'T_TCHL',
       'T_HDL', 'T_LDL', 'T_TG'],
      dtype='object')

In [161]:
base_merged_c = base_merged[['T_ID','T_SEX','T_AGE','T_HTN','T_DM','T_LIP','T_FMFHT1','T_FMFHT2','T_FMFDM1','T_FMFDM2','T_DRINK','T_SMOKE','T_PSM','T_EXER','T_SBP','T_DBP','T_PULSE','T_WAIST','T_HIP','T_HEIGHT','T_WEIGHT','T_BMI','T_GLU0','T_CREATINE','T_AST','T_ALT','T_TCHL','T_HDL','T_LDL','T_TG']]

In [162]:
base_merged_c.replace([66666, 77777, 99999], np.nan, inplace=True)

C:\Users\campus4D044\AppData\Local\Temp\ipykernel_13844\2384870377.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  base_merged_c.replace([66666, 77777, 99999], np.nan, inplace=True)


In [163]:
base_merged_c.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 30 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   T_ID        10000 non-null  object 
 1   T_SEX       10000 non-null  int64  
 2   T_AGE       10000 non-null  int64  
 3   T_HTN       9982 non-null   float64
 4   T_DM        9981 non-null   float64
 5   T_LIP       9978 non-null   float64
 6   T_FMFHT1    10000 non-null  int64  
 7   T_FMFHT2    10000 non-null  int64  
 8   T_FMFDM1    10000 non-null  int64  
 9   T_FMFDM2    10000 non-null  int64  
 10  T_DRINK     9939 non-null   float64
 11  T_SMOKE     9932 non-null   float64
 12  T_PSM       9859 non-null   float64
 13  T_EXER      9944 non-null   float64
 14  T_SBP       9959 non-null   float64
 15  T_DBP       9959 non-null   float64
 16  T_PULSE     9555 non-null   float64
 17  T_WAIST     9868 non-null   float64
 18  T_HIP       9861 non-null   float64
 19  T_HEIGHT    9954 non-null 

In [175]:
base_merged_c = base_merged_c.dropna()

In [14]:
base_merged_c.shape

(8922, 30)

In [15]:
base_merged_c.describe()

T_SEX        T_AGE        T_HTN         T_DM        T_LIP  \
count  8922.000000  8922.000000  8922.000000  8922.000000  8922.000000   
mean      1.660278    53.693679     1.200740     1.068931     1.079466   
std       0.473641     8.701426     0.400576     0.253351     0.270481   
min       1.000000    40.000000     1.000000     1.000000     1.000000   
25%       1.000000    47.000000     1.000000     1.000000     1.000000   
50%       2.000000    53.000000     1.000000     1.000000     1.000000   
75%       2.000000    60.000000     1.000000     1.000000     1.000000   
max       2.000000    86.000000     2.000000     2.000000     2.000000   

          T_FMFHT1     T_FMFHT2     T_FMFDM1     T_FMFDM2      T_DRINK  ...  \
count  8922.000000  8922.000000  8922.000000  8922.000000  8922.000000  ...   
mean      1.103676     1.154001     1.048980     1.081932     1.941157  ...   
std       0.304857     0.360970     0.215838     0.274277     0.974631  ...   
min       1.000000     1.000000     1.000000     1.000000     1.000000  ...   
25%       1.000000     1.000000     1.000000     1.000000     1.000000  ...   
50%       1.000000     1.000000     1.000000     1.000000     1.000000  ...   
75%       1.000000     1.000000     1.000000     1.000000     3.000000  ...   
max       2.000000     2.000000     2.000000     2.000000     3.000000  ...   

          T_WEIGHT        T_BMI       T_GLU0   T_CREATINE        T_AST  \
count  8922.000000  8922.000000  8922.000000  8922.000000  8922.000000   
mean     61.631361    24.008406    94.957072     0.833351    24.228424   
std       9.974941     2.984742    20.605123     0.247405    21.683543   
min      33.000000    15.000000    49.000000     0.390000     5.000000   
25%      54.000000    22.000000    84.000000     0.700000    19.000000   
50%      60.000000    24.000000    91.000000     0.800000    22.000000   
75%      68.000000    26.000000    99.000000     0.920000    26.000000   
max     122.000000    44.000000   394.000000    14.500000  1750.000000   

             T_ALT       T_TCHL        T_HDL        T_LDL         T_TG  
count  8922.000000  8922.000000  8922.000000  8922.000000  8922.000000  
mean     22.586640   196.956512    53.010312   119.566577   121.881641  
std      21.590407    35.425960    13.004768    32.112606    67.326191  
min       6.000000    76.000000    21.000000     2.000000    26.000000  
25%      14.000000   172.000000    44.000000    97.000000    73.000000  
50%      18.000000   195.000000    52.000000   117.000000   105.000000  
75%      25.000000   219.000000    61.000000   139.000000   153.000000  
max     910.000000   402.000000   146.000000   299.000000   400.000000  

[8 rows x 29 columns]

In [114]:
base_merged_c.info()

<class 'pandas.core.frame.DataFrame'>
Index: 6473 entries, 0 to 9999
Data columns (total 30 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   T_ID        6473 non-null   object 
 1   T_SEX       6473 non-null   int64  
 2   T_AGE       6473 non-null   int64  
 3   T_HTN       6473 non-null   float64
 4   T_DM        6473 non-null   float64
 5   T_LIP       6473 non-null   float64
 6   T_FMFHT1    6473 non-null   int64  
 7   T_FMFHT2    6473 non-null   int64  
 8   T_FMFDM1    6473 non-null   int64  
 9   T_FMFDM2    6473 non-null   int64  
 10  T_DRINK     6473 non-null   float64
 11  T_SMOKE     6473 non-null   float64
 12  T_PSM       6473 non-null   float64
 13  T_EXER      6473 non-null   float64
 14  T_SBP       6473 non-null   float64
 15  T_DBP       6473 non-null   float64
 16  T_PULSE     6473 non-null   float64
 17  T_WAIST     6473 non-null   float64
 18  T_HIP       6473 non-null   float64
 19  T_HEIGHT    6473 non-null   floa

In [188]:
def remove_outliers_iqr(df, exclude_cols=[]):
    df_clean = df.copy()
    for col in df_clean.select_dtypes(include=['number']).columns:
        if col in exclude_cols:
            continue  # 제외할 컬럼은 스킵
        Q1 = df_clean[col].quantile(0.25)
        Q3 = df_clean[col].quantile(0.75)
        IQR = Q3 - Q1
        lower = Q1 - 1.5 * IQR
        upper = Q3 + 1.5 * IQR
        df_clean = df_clean[(df_clean[col] >= lower) & (df_clean[col] <= upper)]
    return df_clean

In [189]:
# exclude_cols = ['T_ID','T_SEX','T_AGE','T_HTN','T_DM','T_LIP','T_FMFHT1','T_FMFHT2','T_FMFDM1','T_FMFDM2','T_DRINK','T_SMOKE','T_PSM','T_EXER']
# base_merged_c = remove_outliers_iqr(base_merged_c, exclude_cols=exclude_cols)
# 결측치 제거하지 않아야 성능 향상

In [190]:
base_merged_c.describe()

T_SEX        T_AGE        T_HTN         T_DM        T_LIP  \
count  6473.000000  6473.000000  6473.000000  6473.000000  6473.000000   
mean      1.705855    53.262320     1.167465     1.030125     1.070601   
std       0.455693     8.617423     0.373419     0.170945     0.256177   
min       1.000000    40.000000     1.000000     1.000000     1.000000   
25%       1.000000    46.000000     1.000000     1.000000     1.000000   
50%       2.000000    52.000000     1.000000     1.000000     1.000000   
75%       2.000000    60.000000     1.000000     1.000000     1.000000   
max       2.000000    86.000000     2.000000     2.000000     2.000000   

          T_FMFHT1     T_FMFHT2     T_FMFDM1     T_FMFDM2      T_DRINK  ...  \
count  6473.000000  6473.000000  6473.000000  6473.000000  6473.000000  ...   
mean      1.104897     1.155878     1.046655     1.080643     1.899274  ...   
std       0.306445     0.362768     0.210916     0.272307     0.974039  ...   
min       1.000000     1.000000     1.000000     1.000000     1.000000  ...   
25%       1.000000     1.000000     1.000000     1.000000     1.000000  ...   
50%       1.000000     1.000000     1.000000     1.000000     1.000000  ...   
75%       1.000000     1.000000     1.000000     1.000000     3.000000  ...   
max       2.000000     2.000000     2.000000     2.000000     3.000000  ...   

          T_WEIGHT        T_BMI       T_GLU0   T_CREATINE        T_AST  \
count  6473.000000  6473.000000  6473.000000  6473.000000  6473.000000   
mean     60.250116    23.628457    90.527576     0.810113    21.501159   
std       8.765504     2.673967     9.852194     0.152060     4.826890   
min      36.000000    16.000000    62.000000     0.400000     7.000000   
25%      54.000000    22.000000    84.000000     0.700000    18.000000   
50%      59.000000    23.000000    90.000000     0.800000    21.000000   
75%      66.000000    25.000000    96.000000     0.900000    24.000000   
max      86.000000    32.000000   121.000000     1.200000    38.000000   

             T_ALT       T_TCHL        T_HDL        T_LDL         T_TG  
count  6473.000000  6473.000000  6473.000000  6473.000000  6473.000000  
mean     18.147227   193.977599    53.690097   118.798393   107.429785  
std       6.355612    32.178479    11.909361    29.080346    49.227861  
min       6.000000   103.000000    23.000000    40.000000    26.000000  
25%      13.000000   171.000000    45.000000    98.000000    69.000000  
50%      17.000000   193.000000    53.000000   117.000000    97.000000  
75%      22.000000   215.000000    61.000000   138.000000   137.000000  
max      36.000000   287.000000    87.000000   198.000000   252.000000  

[8 rows x 29 columns]

In [195]:
base_merged_x = base_merged_c.drop(['T_ID','T_HTN','T_DM','T_LIP'],axis=1)
base_merged_y = base_merged_c[['T_HTN','T_DM','T_LIP']]
base_merged_htn = base_merged_c[['T_HTN']]
base_merged_dm = base_merged_c[['T_DM']]
base_merged_lip = base_merged_c[['T_LIP']]

base_merged_htn = base_merged_htn.replace({1.0: 0, 2.0: 1})
base_merged_dm = base_merged_dm.replace({1.0: 0, 2.0: 1})
base_merged_lip = base_merged_lip.replace({1.0: 0, 2.0: 1})

### HTN 예측

In [1244]:
tr_x, tt_x, tr_y, tt_y = train_test_split(base_merged_x, base_merged_htn, train_size=0.7, random_state=10, stratify=base_merged_htn)
tr_x.shape, tt_x.shape, tr_y.shape, tt_y.shape

((4531, 26), (1942, 26), (4531, 1), (1942, 1))

In [1245]:
smote1 = SMOTE(random_state=10)
r_tr_x, r_tr_y = smote1.fit_resample(tr_x, tr_y)

In [1246]:
ss1 = StandardScaler().fit(r_tr_x)
s_tr_x = ss1.transform(r_tr_x)
s_tt_x = ss1.transform(tt_x)

In [1247]:
model1 = DecisionTreeClassifier(random_state=10).fit(s_tr_x,r_tr_y)
model2 = RandomForestClassifier(n_estimators=100,max_depth=None,class_weight='balanced',random_state=10).fit(s_tr_x,r_tr_y)
model3 = GradientBoostingClassifier(n_estimators=100,learning_rate=0.1,max_depth=3,random_state=10).fit(s_tr_x,r_tr_y)
model4 = SVC(kernel='rbf', C=1.0, gamma='scale', random_state=10).fit(s_tr_x,r_tr_y)

C:\Users\campus4D044\anaconda3\Lib\site-packages\sklearn\base.py:1473: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
C:\Users\campus4D044\anaconda3\Lib\site-packages\sklearn\preprocessing\_label.py:114: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\campus4D044\anaconda3\Lib\site-packages\sklearn\utils\validation.py:1339: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [1248]:
print(classification_report(tt_y, model1.predict(s_tt_x)))
print()
print(classification_report(tt_y, model2.predict(s_tt_x)))
print()
print(classification_report(tt_y, model3.predict(s_tt_x)))
print()
print(classification_report(tt_y, model4.predict(s_tt_x)))

              precision    recall  f1-score   support

         0.0       0.87      0.82      0.84      1617
         1.0       0.31      0.41      0.36       325

    accuracy                           0.75      1942
   macro avg       0.59      0.61      0.60      1942
weighted avg       0.78      0.75      0.76      1942


              precision    recall  f1-score   support

         0.0       0.87      0.93      0.90      1617
         1.0       0.47      0.32      0.38       325

    accuracy                           0.83      1942
   macro avg       0.67      0.62      0.64      1942
weighted avg       0.80      0.83      0.81      1942


              precision    recall  f1-score   support

         0.0       0.88      0.91      0.89      1617
         1.0       0.45      0.37      0.41       325

    accuracy                           0.82      1942
   macro avg       0.67      0.64      0.65      1942
weighted avg       0.81      0.82      0.81      1942


              pr

In [1249]:
model5 = XGBClassifier(n_estimators=100,learning_rate=0.1,max_depth=3,scale_pos_weight=4,random_state=10,use_label_encoder=False,eval_metric='logloss').fit(s_tr_x,r_tr_y)
model6 = XGBClassifier(random_state=10).fit(s_tr_x,r_tr_y)

C:\Users\campus4D044\anaconda3\Lib\site-packages\xgboost\training.py:183: UserWarning: [15:59:11] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


In [1250]:
print(classification_report(tt_y, model5.predict(s_tt_x)))
print(classification_report(tt_y, model6.predict(s_tt_x)))

              precision    recall  f1-score   support

         0.0       0.94      0.65      0.77      1617
         1.0       0.32      0.80      0.45       325

    accuracy                           0.68      1942
   macro avg       0.63      0.73      0.61      1942
weighted avg       0.84      0.68      0.72      1942

              precision    recall  f1-score   support

         0.0       0.87      0.94      0.90      1617
         1.0       0.51      0.32      0.40       325

    accuracy                           0.84      1942
   macro avg       0.69      0.63      0.65      1942
weighted avg       0.81      0.84      0.82      1942



### DM 예측

In [92]:
tr_x, tt_x, tr_y, tt_y = train_test_split(base_merged_x, base_merged_dm, train_size=0.7, random_state=10, stratify=base_merged_dm)
tr_x.shape, tt_x.shape, tr_y.shape, tt_y.shape

((6245, 17), (2677, 17), (6245, 1), (2677, 1))

In [93]:
smote2 = SMOTE(random_state=10)
r_tr_x, r_tr_y = smote2.fit_resample(tr_x, tr_y)

In [94]:
ss2 = StandardScaler().fit(r_tr_x)
s_tr_x = ss2.transform(r_tr_x)
s_tt_x = ss2.transform(tt_x)

In [95]:
model1 = DecisionTreeClassifier(random_state=10).fit(s_tr_x,r_tr_y)
model2 = RandomForestClassifier(n_estimators=100,max_depth=None,class_weight='balanced',random_state=10).fit(s_tr_x,r_tr_y)
model3 = GradientBoostingClassifier(n_estimators=100,learning_rate=0.1,max_depth=3,random_state=10).fit(s_tr_x,r_tr_y)
model4 = SVC(kernel='rbf', C=1.0, gamma='scale', random_state=10).fit(s_tr_x,r_tr_y)
model5 = XGBClassifier(random_state=10).fit(s_tr_x,r_tr_y)

C:\Users\campus4D044\anaconda3\Lib\site-packages\sklearn\base.py:1473: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
C:\Users\campus4D044\anaconda3\Lib\site-packages\sklearn\preprocessing\_label.py:114: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\campus4D044\anaconda3\Lib\site-packages\sklearn\utils\validation.py:1339: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [96]:
print('Decision Tree Classifier')
print(classification_report(tt_y, model1.predict(s_tt_x)))
print()
print('Random Forest Classifier')
print(classification_report(tt_y, model2.predict(s_tt_x)))
print()
print('Gradient Boosting Classifier')
print(classification_report(tt_y, model3.predict(s_tt_x)))
print()
print('Support Vector Classifier')
print(classification_report(tt_y, model4.predict(s_tt_x)))
print()
print('XG Booster Classifier')
print(classification_report(tt_y, model5.predict(s_tt_x)))

Decision Tree Classifier
              precision    recall  f1-score   support

         0.0       0.96      0.92      0.94      2492
         1.0       0.31      0.48      0.38       185

    accuracy                           0.89      2677
   macro avg       0.63      0.70      0.66      2677
weighted avg       0.91      0.89      0.90      2677


Random Forest Classifier
              precision    recall  f1-score   support

         0.0       0.97      0.97      0.97      2492
         1.0       0.57      0.60      0.59       185

    accuracy                           0.94      2677
   macro avg       0.77      0.78      0.78      2677
weighted avg       0.94      0.94      0.94      2677


Gradient Boosting Classifier
              precision    recall  f1-score   support

         0.0       0.97      0.95      0.96      2492
         1.0       0.49      0.64      0.55       185

    accuracy                           0.93      2677
   macro avg       0.73      0.79      0.76    

In [97]:
print('Decision Tree Classifier')
print(f1_score(tt_y, model1.predict(s_tt_x)))
print()
print('Random Forest Classifier')
print(f1_score(tt_y, model2.predict(s_tt_x)))
print()
print('Gradient Boosting Classifier')
print(f1_score(tt_y, model3.predict(s_tt_x)))
print()
print('Support Vector Classifier')
print(f1_score(tt_y, model4.predict(s_tt_x)))
print()
print('XG Booster Classifier')
print(f1_score(tt_y, model5.predict(s_tt_x)))

Decision Tree Classifier
0.3755274261603376

Random Forest Classifier
0.5857519788918206

Gradient Boosting Classifier
0.5539906103286385

Support Vector Classifier
0.5062761506276151

XG Booster Classifier
0.5757575757575758


In [23]:
model5 = XGBClassifier(n_estimators=100,learning_rate=0.1,max_depth=3,scale_pos_weight=4,random_state=10,use_label_encoder=False,eval_metric='logloss').fit(s_tr_x,r_tr_y)
model6 = XGBClassifier(random_state=10).fit(s_tr_x,r_tr_y)

C:\Users\campus4D044\anaconda3\Lib\site-packages\xgboost\training.py:183: UserWarning: [19:23:58] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


In [24]:
print(classification_report(tt_y, model5.predict(s_tt_x)))
print(classification_report(tt_y, model6.predict(s_tt_x)))

              precision    recall  f1-score   support

         0.0       0.98      0.90      0.94      2492
         1.0       0.37      0.78      0.50       185

    accuracy                           0.89      2677
   macro avg       0.68      0.84      0.72      2677
weighted avg       0.94      0.89      0.91      2677

              precision    recall  f1-score   support

         0.0       0.96      0.98      0.97      2492
         1.0       0.61      0.45      0.52       185

    accuracy                           0.94      2677
   macro avg       0.78      0.71      0.74      2677
weighted avg       0.94      0.94      0.94      2677



### LIP 예측

In [1258]:
tr_x, tt_x, tr_y, tt_y = train_test_split(base_merged_x, base_merged_lip, train_size=0.7, random_state=10, stratify=base_merged_lip)
tr_x.shape, tt_x.shape, tr_y.shape, tt_y.shape

((4531, 26), (1942, 26), (4531, 1), (1942, 1))

In [1259]:
smote3 = SMOTE(random_state=10)
r_tr_x, r_tr_y = smote3.fit_resample(tr_x, tr_y)

In [1260]:
ss3 = StandardScaler().fit(r_tr_x)
s_tr_x = ss3.transform(r_tr_x)
s_tt_x = ss3.transform(tt_x)

In [1261]:
model1 = DecisionTreeClassifier(random_state=10).fit(s_tr_x,r_tr_y)
model2 = RandomForestClassifier(n_estimators=100,max_depth=None,class_weight='balanced',random_state=10).fit(s_tr_x,r_tr_y)
model3 = GradientBoostingClassifier(n_estimators=100,learning_rate=0.1,max_depth=3,random_state=10).fit(s_tr_x,r_tr_y)
model4 = SVC(kernel='rbf', C=1.0, gamma='scale', random_state=10).fit(s_tr_x,r_tr_y)

C:\Users\campus4D044\anaconda3\Lib\site-packages\sklearn\base.py:1473: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
C:\Users\campus4D044\anaconda3\Lib\site-packages\sklearn\preprocessing\_label.py:114: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\campus4D044\anaconda3\Lib\site-packages\sklearn\utils\validation.py:1339: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [1262]:
print(classification_report(tt_y, model1.predict(s_tt_x)))
print()
print(classification_report(tt_y, model2.predict(s_tt_x)))
print()
print(classification_report(tt_y, model3.predict(s_tt_x)))
print()
print(classification_report(tt_y, model4.predict(s_tt_x)))

              precision    recall  f1-score   support

         0.0       0.94      0.90      0.92      1805
         1.0       0.12      0.18      0.15       137

    accuracy                           0.85      1942
   macro avg       0.53      0.54      0.53      1942
weighted avg       0.88      0.85      0.86      1942


              precision    recall  f1-score   support

         0.0       0.93      1.00      0.96      1805
         1.0       0.12      0.01      0.01       137

    accuracy                           0.93      1942
   macro avg       0.53      0.50      0.49      1942
weighted avg       0.87      0.93      0.89      1942


              precision    recall  f1-score   support

         0.0       0.93      0.98      0.95      1805
         1.0       0.10      0.03      0.04       137

    accuracy                           0.91      1942
   macro avg       0.51      0.50      0.50      1942
weighted avg       0.87      0.91      0.89      1942


              pr

In [1263]:
model5 = XGBClassifier(n_estimators=100,learning_rate=0.1,max_depth=3,scale_pos_weight=4,random_state=10,use_label_encoder=False,eval_metric='logloss').fit(s_tr_x,r_tr_y)
model6 = XGBClassifier(random_state=10).fit(s_tr_x,r_tr_y)

C:\Users\campus4D044\anaconda3\Lib\site-packages\xgboost\training.py:183: UserWarning: [15:59:33] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


In [1264]:
print(classification_report(tt_y, model5.predict(s_tt_x)))
print(classification_report(tt_y, model6.predict(s_tt_x)))

              precision    recall  f1-score   support

         0.0       0.94      0.77      0.85      1805
         1.0       0.12      0.39      0.18       137

    accuracy                           0.75      1942
   macro avg       0.53      0.58      0.52      1942
weighted avg       0.89      0.75      0.80      1942

              precision    recall  f1-score   support

         0.0       0.93      0.99      0.96      1805
         1.0       0.15      0.03      0.05       137

    accuracy                           0.92      1942
   macro avg       0.54      0.51      0.50      1942
weighted avg       0.88      0.92      0.89      1942



### 피쳐 추출

In [169]:
base_merged_r = base_merged_c[['T_ID','T_HTN','T_DM','T_LIP','T_SEX','T_AGE','T_HEIGHT','T_WEIGHT','T_WAIST','T_SBP','T_DBP','T_GLU0','T_TCHL','T_TG','T_HDL','T_LDL','T_CREATINE','T_AST','T_ALT','T_SMOKE','T_DRINK']]

In [170]:
base_merged_r = base_merged_c.copy()

In [171]:
base_merged_x = base_merged_r.drop(['T_ID','T_HTN','T_DM','T_LIP'],axis=1)
base_merged_y = base_merged_r[['T_HTN','T_DM','T_LIP']]
base_merged_htn = base_merged_r[['T_HTN']]
base_merged_dm = base_merged_r[['T_DM']]
base_merged_lip = base_merged_r[['T_LIP']]

base_merged_htn = base_merged_htn.replace({1.0: 0, 2.0: 1})
base_merged_dm = base_merged_dm.replace({1.0: 0, 2.0: 1})
base_merged_lip = base_merged_lip.replace({1.0: 0, 2.0: 1})

### HTN 예측

In [1267]:
tr_x, tt_x, tr_y, tt_y = train_test_split(base_merged_x, base_merged_htn, train_size=0.7, random_state=10, stratify=base_merged_htn)
tr_x.shape, tt_x.shape, tr_y.shape, tt_y.shape

((4531, 17), (1942, 17), (4531, 1), (1942, 1))

In [1268]:
smote1 = SMOTE(random_state=10)
r_tr_x, r_tr_y = smote1.fit_resample(tr_x, tr_y)

In [1269]:
ss1 = StandardScaler().fit(r_tr_x)
s_tr_x = ss1.transform(r_tr_x)
s_tt_x = ss1.transform(tt_x)

In [1270]:
model1 = DecisionTreeClassifier(random_state=10).fit(s_tr_x,r_tr_y)
model2 = RandomForestClassifier(n_estimators=100,max_depth=None,class_weight='balanced',random_state=10).fit(s_tr_x,r_tr_y)
model3 = GradientBoostingClassifier(n_estimators=100,learning_rate=0.1,max_depth=3,random_state=10).fit(s_tr_x,r_tr_y)
model4 = SVC(kernel='rbf', C=1.0, gamma='scale', random_state=10).fit(s_tr_x,r_tr_y)

C:\Users\campus4D044\anaconda3\Lib\site-packages\sklearn\base.py:1473: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
C:\Users\campus4D044\anaconda3\Lib\site-packages\sklearn\preprocessing\_label.py:114: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\campus4D044\anaconda3\Lib\site-packages\sklearn\utils\validation.py:1339: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [1271]:
print(classification_report(tt_y, model1.predict(s_tt_x)))
print()
print(classification_report(tt_y, model2.predict(s_tt_x)))
print()
print(classification_report(tt_y, model3.predict(s_tt_x)))
print()
print(classification_report(tt_y, model4.predict(s_tt_x)))

              precision    recall  f1-score   support

         0.0       0.87      0.81      0.84      1617
         1.0       0.31      0.42      0.35       325

    accuracy                           0.74      1942
   macro avg       0.59      0.61      0.60      1942
weighted avg       0.78      0.74      0.76      1942


              precision    recall  f1-score   support

         0.0       0.87      0.90      0.89      1617
         1.0       0.41      0.35      0.38       325

    accuracy                           0.81      1942
   macro avg       0.64      0.63      0.63      1942
weighted avg       0.80      0.81      0.80      1942


              precision    recall  f1-score   support

         0.0       0.89      0.87      0.88      1617
         1.0       0.41      0.47      0.44       325

    accuracy                           0.80      1942
   macro avg       0.65      0.67      0.66      1942
weighted avg       0.81      0.80      0.80      1942


              pr

In [1272]:
model5 = XGBClassifier(n_estimators=100,learning_rate=0.1,max_depth=3,scale_pos_weight=4,random_state=10,use_label_encoder=False,eval_metric='logloss').fit(s_tr_x,r_tr_y)
model6 = XGBClassifier(random_state=10).fit(s_tr_x,r_tr_y)

C:\Users\campus4D044\anaconda3\Lib\site-packages\xgboost\training.py:183: UserWarning: [15:59:39] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


In [1273]:
print(classification_report(tt_y, model5.predict(s_tt_x)))
print(classification_report(tt_y, model6.predict(s_tt_x)))

              precision    recall  f1-score   support

         0.0       0.95      0.60      0.74      1617
         1.0       0.30      0.83      0.44       325

    accuracy                           0.64      1942
   macro avg       0.62      0.72      0.59      1942
weighted avg       0.84      0.64      0.69      1942

              precision    recall  f1-score   support

         0.0       0.87      0.92      0.89      1617
         1.0       0.44      0.29      0.35       325

    accuracy                           0.82      1942
   macro avg       0.65      0.61      0.62      1942
weighted avg       0.79      0.82      0.80      1942



In [1274]:
print(f1_score(tt_y, model1.predict(s_tt_x)))
print(f1_score(tt_y, model2.predict(s_tt_x)))
print(f1_score(tt_y, model3.predict(s_tt_x)))
print(f1_score(tt_y, model4.predict(s_tt_x)))
print(f1_score(tt_y, model5.predict(s_tt_x)))
print(f1_score(tt_y, model6.predict(s_tt_x)))

0.3520208604954368
0.3814262023217247
0.4396551724137931
0.43105446118192353
0.4381568310428456
0.34990791896869244


In [1275]:
grid_model1 = RandomForestClassifier()

grid_model1_p = {
    'n_estimators': [100, 200],
    'max_depth': [None, 10, 20],
    'min_samples_split': [2, 5],
    'min_samples_leaf': [1, 2],
    'max_features': ['sqrt', 'log2']
}

grid_model1 = GridSearchCV(grid_model1, grid_model1_p, cv=3, scoring='f1', n_jobs=-1)
grid_model1.fit(s_tr_x, r_tr_y)

print(grid_model1.best_params_)
print(grid_model1.best_score_)

C:\Users\campus4D044\anaconda3\Lib\site-packages\sklearn\base.py:1473: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


{'max_depth': None, 'max_features': 'sqrt', 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 200}
0.8778211755229598


In [1276]:
grid_model2 = GradientBoostingClassifier()

grid_model2_p = {
    'n_estimators': [100, 200],
    'learning_rate': [0.01, 0.1, 0.3],
    'max_depth': [3, 5],
    'subsample': [0.8, 1]
}

grid_model2 = GridSearchCV(grid_model2, grid_model2_p, cv=3, scoring='f1', n_jobs=-1)
grid_model2.fit(s_tr_x, r_tr_y)

print(grid_model2.best_params_)
print(grid_model2.best_score_)

C:\Users\campus4D044\anaconda3\Lib\site-packages\sklearn\preprocessing\_label.py:114: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


{'learning_rate': 0.1, 'max_depth': 5, 'n_estimators': 100, 'subsample': 1}
0.8272519026810602


In [1277]:
grid_model3 = XGBClassifier(use_label_encoder=False, eval_metric='logloss')

grid_model3_p = {
    'n_estimators': [100, 200],
    'max_depth': [3, 5, 7],
    'learning_rate': [0.01, 0.1, 0.3],
    'subsample': [0.8, 1],
    'colsample_bytree': [0.8, 1]
}

grid_model3 = GridSearchCV(grid_model3, grid_model3_p, cv=3, scoring='f1', n_jobs=-1)
grid_model3.fit(s_tr_x, r_tr_y)

print(grid_model3.best_params_)
print(grid_model3.best_score_)

{'colsample_bytree': 1, 'learning_rate': 0.1, 'max_depth': 7, 'n_estimators': 100, 'subsample': 1}
0.8398621814448356


C:\Users\campus4D044\anaconda3\Lib\site-packages\xgboost\training.py:183: UserWarning: [16:01:37] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


In [1278]:
grid_model4 = SVC()

grid_model4_p = {
    'C': [0.1, 1, 10],
    'kernel': ['linear', 'rbf'],
    'gamma': ['scale', 'auto', 0.01, 0.001]
}

grid_model4 = GridSearchCV(grid_model4, grid_model4_p, cv=3, scoring='accuracy', n_jobs=-1)
grid_model4.fit(s_tr_x, r_tr_y)

print(grid_model4.best_params_)
print(grid_model4.best_score_)

C:\Users\campus4D044\anaconda3\Lib\site-packages\sklearn\utils\validation.py:1339: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


{'C': 10, 'gamma': 'auto', 'kernel': 'rbf'}
0.8667827034082959


In [1279]:
grid_model1 = RandomForestClassifier(max_depth=None, max_features='sqrt', min_samples_leaf=1, min_samples_split=2, n_estimators=100).fit(s_tr_x, r_tr_y)
grid_model2 = GradientBoostingClassifier(learning_rate=0.01, max_depth=3, n_estimators=200, subsample=0.8).fit(s_tr_x, r_tr_y)
grid_model3 = XGBClassifier(use_label_encoder=False, eval_metric='logloss', colsample_bytree=1, learning_rate=0.01, max_depth=7, n_estimators=200, subsample=0.8).fit(s_tr_x, r_tr_y)
grid_model4 = SVC(C=10, gamma='scale', kernel='rbf').fit(s_tr_x, r_tr_y)

C:\Users\campus4D044\anaconda3\Lib\site-packages\sklearn\base.py:1473: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
C:\Users\campus4D044\anaconda3\Lib\site-packages\sklearn\preprocessing\_label.py:114: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\campus4D044\anaconda3\Lib\site-packages\xgboost\training.py:183: UserWarning: [16:02:13] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
C:\Users\campus4D044\anaconda3\Lib\site-packages\sklearn\utils\validation.py:1339: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape

In [1280]:
print(f1_score(tt_y, model1.predict(s_tt_x)))
print(f1_score(tt_y, model2.predict(s_tt_x)))
print(f1_score(tt_y, model3.predict(s_tt_x)))
print(f1_score(tt_y, model4.predict(s_tt_x)))

0.3520208604954368
0.3814262023217247
0.4396551724137931
0.43105446118192353


### DM 예측

In [98]:
tr_x, tt_x, tr_y, tt_y = train_test_split(base_merged_x, base_merged_dm, train_size=0.7, random_state=10, stratify=base_merged_dm)
tr_x.shape, tt_x.shape, tr_y.shape, tt_y.shape

((6245, 17), (2677, 17), (6245, 1), (2677, 1))

In [99]:
smote2 = SMOTE(random_state=10)
r_tr_x, r_tr_y = smote2.fit_resample(tr_x, tr_y)

In [100]:
ss2 = StandardScaler().fit(r_tr_x)
s_tr_x = ss2.transform(r_tr_x)
s_tt_x = ss2.transform(tt_x)

In [101]:
model1 = DecisionTreeClassifier(random_state=10).fit(s_tr_x,r_tr_y)
model2 = RandomForestClassifier(n_estimators=100,max_depth=None,class_weight='balanced',random_state=10).fit(s_tr_x,r_tr_y)
model3 = GradientBoostingClassifier(n_estimators=100,learning_rate=0.1,max_depth=3,random_state=10).fit(s_tr_x,r_tr_y)
model4 = SVC(kernel='rbf', C=1.0, gamma='scale', random_state=10).fit(s_tr_x,r_tr_y)
model5 = XGBClassifier(random_state=10).fit(s_tr_x,r_tr_y)

C:\Users\campus4D044\anaconda3\Lib\site-packages\sklearn\base.py:1473: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
C:\Users\campus4D044\anaconda3\Lib\site-packages\sklearn\preprocessing\_label.py:114: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\campus4D044\anaconda3\Lib\site-packages\sklearn\utils\validation.py:1339: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [102]:
print('Decision Tree Classifier')
print(classification_report(tt_y, model1.predict(s_tt_x)))
print()
print('Random Forest Classifier')
print(classification_report(tt_y, model2.predict(s_tt_x)))
print()
print('Gradient Boosting Classifier')
print(classification_report(tt_y, model3.predict(s_tt_x)))
print()
print('Support Vector Classifier')
print(classification_report(tt_y, model4.predict(s_tt_x)))
print()
print('XG Booster Classifier')
print(classification_report(tt_y, model5.predict(s_tt_x)))

Decision Tree Classifier
              precision    recall  f1-score   support

         0.0       0.96      0.92      0.94      2492
         1.0       0.31      0.48      0.38       185

    accuracy                           0.89      2677
   macro avg       0.63      0.70      0.66      2677
weighted avg       0.91      0.89      0.90      2677


Random Forest Classifier
              precision    recall  f1-score   support

         0.0       0.97      0.97      0.97      2492
         1.0       0.57      0.60      0.59       185

    accuracy                           0.94      2677
   macro avg       0.77      0.78      0.78      2677
weighted avg       0.94      0.94      0.94      2677


Gradient Boosting Classifier
              precision    recall  f1-score   support

         0.0       0.97      0.95      0.96      2492
         1.0       0.49      0.64      0.55       185

    accuracy                           0.93      2677
   macro avg       0.73      0.79      0.76    

In [103]:
print('Decision Tree Classifier')
print(f1_score(tt_y, model1.predict(s_tt_x)))
print()
print('Random Forest Classifier')
print(f1_score(tt_y, model2.predict(s_tt_x)))
print()
print('Gradient Boosting Classifier')
print(f1_score(tt_y, model3.predict(s_tt_x)))
print()
print('Support Vector Classifier')
print(f1_score(tt_y, model4.predict(s_tt_x)))
print()
print('XG Booster Classifier')
print(f1_score(tt_y, model5.predict(s_tt_x)))

Decision Tree Classifier
0.3755274261603376

Random Forest Classifier
0.5857519788918206

Gradient Boosting Classifier
0.5539906103286385

Support Vector Classifier
0.5062761506276151

XG Booster Classifier
0.5757575757575758


In [1286]:
model5 = XGBClassifier(n_estimators=100,learning_rate=0.1,max_depth=3,scale_pos_weight=4,random_state=10,use_label_encoder=False,eval_metric='logloss').fit(s_tr_x,r_tr_y)
model6 = XGBClassifier(random_state=10).fit(s_tr_x,r_tr_y)

C:\Users\campus4D044\anaconda3\Lib\site-packages\xgboost\training.py:183: UserWarning: [16:02:22] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


In [1287]:
print(classification_report(tt_y, model5.predict(s_tt_x)))
print(classification_report(tt_y, model6.predict(s_tt_x)))

              precision    recall  f1-score   support

         0.0       0.99      0.85      0.91      1883
         1.0       0.12      0.64      0.20        59

    accuracy                           0.84      1942
   macro avg       0.55      0.75      0.55      1942
weighted avg       0.96      0.84      0.89      1942

              precision    recall  f1-score   support

         0.0       0.97      0.99      0.98      1883
         1.0       0.30      0.15      0.20        59

    accuracy                           0.96      1942
   macro avg       0.64      0.57      0.59      1942
weighted avg       0.95      0.96      0.96      1942



In [1288]:
print(f1_score(tt_y, model1.predict(s_tt_x)))
print(f1_score(tt_y, model2.predict(s_tt_x)))
print(f1_score(tt_y, model3.predict(s_tt_x)))
print(f1_score(tt_y, model4.predict(s_tt_x)))
print(f1_score(tt_y, model5.predict(s_tt_x)))
print(f1_score(tt_y, model6.predict(s_tt_x)))

0.12244897959183673
0.13333333333333333
0.2641509433962264
0.2
0.19638242894056848
0.20224719101123595


In [1289]:
grid_model1 = RandomForestClassifier()

grid_model1_p = {
    'n_estimators': [100, 200],
    'max_depth': [None, 10, 20],
    'min_samples_split': [2, 5],
    'min_samples_leaf': [1, 2],
    'max_features': ['sqrt', 'log2']
}

grid_model1 = GridSearchCV(grid_model1, grid_model1_p, cv=3, scoring='f1', n_jobs=-1)
grid_model1.fit(s_tr_x, r_tr_y)

print(grid_model1.best_params_)
print(grid_model1.best_score_)

C:\Users\campus4D044\anaconda3\Lib\site-packages\sklearn\base.py:1473: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


{'max_depth': None, 'max_features': 'log2', 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 100}
0.9810907775672307


In [1290]:
grid_model2 = GradientBoostingClassifier()

grid_model2_p = {
    'n_estimators': [100, 200],
    'learning_rate': [0.01, 0.1, 0.3],
    'max_depth': [3, 5],
    'subsample': [0.8, 1]
}

grid_model2 = GridSearchCV(grid_model2, grid_model2_p, cv=3, scoring='f1', n_jobs=-1)
grid_model2.fit(s_tr_x, r_tr_y)

print(grid_model2.best_params_)
print(grid_model2.best_score_)

C:\Users\campus4D044\anaconda3\Lib\site-packages\sklearn\preprocessing\_label.py:114: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


{'learning_rate': 0.3, 'max_depth': 5, 'n_estimators': 200, 'subsample': 1}
0.9827633722791301


In [1291]:
grid_model3 = XGBClassifier(use_label_encoder=False, eval_metric='logloss')

grid_model3_p = {
    'n_estimators': [100, 200],
    'max_depth': [3, 5, 7],
    'learning_rate': [0.01, 0.1, 0.3],
    'subsample': [0.8, 1],
    'colsample_bytree': [0.8, 1]
}

grid_model3 = GridSearchCV(grid_model3, grid_model3_p, cv=3, scoring='f1', n_jobs=-1)
grid_model3.fit(s_tr_x, r_tr_y)

print(grid_model3.best_params_)
print(grid_model3.best_score_)

C:\Users\campus4D044\anaconda3\Lib\site-packages\xgboost\training.py:183: UserWarning: [16:04:37] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


{'colsample_bytree': 0.8, 'learning_rate': 0.1, 'max_depth': 7, 'n_estimators': 200, 'subsample': 1}
0.985438891999809


In [1292]:
grid_model4 = SVC()

grid_model4_p = {
    'C': [0.1, 1, 10],
    'kernel': ['linear', 'rbf'],
    'gamma': ['scale', 'auto', 0.01, 0.001]
}

grid_model4 = GridSearchCV(grid_model4, grid_model4_p, cv=3, scoring='accuracy', n_jobs=-1)
grid_model4.fit(s_tr_x, r_tr_y)

print(grid_model4.best_params_)
print(grid_model4.best_score_)

C:\Users\campus4D044\anaconda3\Lib\site-packages\sklearn\utils\validation.py:1339: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


{'C': 10, 'gamma': 'scale', 'kernel': 'rbf'}
0.981683731513083


In [1293]:
grid_model1 = RandomForestClassifier(max_depth=None, max_features='log2', min_samples_leaf=1, min_samples_split=2, n_estimators=100).fit(s_tr_x, r_tr_y)
grid_model2 = GradientBoostingClassifier(learning_rate=0.3, max_depth=5, n_estimators=200, subsample=1).fit(s_tr_x, r_tr_y)
grid_model3 = XGBClassifier(use_label_encoder=False, eval_metric='logloss', colsample_bytree=0.8, learning_rate=0.1, max_depth=7, n_estimators=200, subsample=1).fit(s_tr_x, r_tr_y)
grid_model4 = SVC(C=10, gamma='auto', kernel='rbf').fit(s_tr_x, r_tr_y)

C:\Users\campus4D044\anaconda3\Lib\site-packages\sklearn\base.py:1473: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
C:\Users\campus4D044\anaconda3\Lib\site-packages\sklearn\preprocessing\_label.py:114: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\campus4D044\anaconda3\Lib\site-packages\xgboost\training.py:183: UserWarning: [16:05:21] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
C:\Users\campus4D044\anaconda3\Lib\site-packages\sklearn\utils\validation.py:1339: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape

In [1294]:
print(f1_score(tt_y, model1.predict(s_tt_x)))
print(f1_score(tt_y, model2.predict(s_tt_x)))
print(f1_score(tt_y, model3.predict(s_tt_x)))
print(f1_score(tt_y, model4.predict(s_tt_x)))

0.12244897959183673
0.13333333333333333
0.2641509433962264
0.2


### LIP 예측

In [1295]:
tr_x, tt_x, tr_y, tt_y = train_test_split(base_merged_x, base_merged_lip, train_size=0.7, random_state=10, stratify=base_merged_lip)
tr_x.shape, tt_x.shape, tr_y.shape, tt_y.shape

((4531, 17), (1942, 17), (4531, 1), (1942, 1))

In [1296]:
smote3 = SMOTE(random_state=10)
r_tr_x, r_tr_y = smote3.fit_resample(tr_x, tr_y)

In [1297]:
ss3 = StandardScaler().fit(r_tr_x)
s_tr_x = ss3.transform(r_tr_x)
s_tt_x = ss3.transform(tt_x)

In [1298]:
model1 = DecisionTreeClassifier(random_state=10).fit(s_tr_x,r_tr_y)
model2 = RandomForestClassifier(n_estimators=100,max_depth=None,class_weight='balanced',random_state=10).fit(s_tr_x,r_tr_y)
model3 = GradientBoostingClassifier(n_estimators=100,learning_rate=0.1,max_depth=3,random_state=10).fit(s_tr_x,r_tr_y)
model4 = SVC(kernel='rbf', C=1.0, gamma='scale', random_state=10).fit(s_tr_x,r_tr_y)

C:\Users\campus4D044\anaconda3\Lib\site-packages\sklearn\base.py:1473: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
C:\Users\campus4D044\anaconda3\Lib\site-packages\sklearn\preprocessing\_label.py:114: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\campus4D044\anaconda3\Lib\site-packages\sklearn\utils\validation.py:1339: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [1299]:
print(classification_report(tt_y, model1.predict(s_tt_x)))
print()
print(classification_report(tt_y, model2.predict(s_tt_x)))
print()
print(classification_report(tt_y, model3.predict(s_tt_x)))
print()
print(classification_report(tt_y, model4.predict(s_tt_x)))

              precision    recall  f1-score   support

         0.0       0.94      0.88      0.90      1805
         1.0       0.12      0.22      0.15       137

    accuracy                           0.83      1942
   macro avg       0.53      0.55      0.53      1942
weighted avg       0.88      0.83      0.85      1942


              precision    recall  f1-score   support

         0.0       0.93      0.98      0.96      1805
         1.0       0.23      0.08      0.12       137

    accuracy                           0.92      1942
   macro avg       0.58      0.53      0.54      1942
weighted avg       0.88      0.92      0.90      1942


              precision    recall  f1-score   support

         0.0       0.94      0.93      0.93      1805
         1.0       0.16      0.18      0.17       137

    accuracy                           0.87      1942
   macro avg       0.55      0.55      0.55      1942
weighted avg       0.88      0.87      0.88      1942


              pr

In [1300]:
model5 = XGBClassifier(n_estimators=100,learning_rate=0.1,max_depth=3,scale_pos_weight=4,random_state=10,use_label_encoder=False,eval_metric='logloss').fit(s_tr_x,r_tr_y)
model6 = XGBClassifier(random_state=10).fit(s_tr_x,r_tr_y)

C:\Users\campus4D044\anaconda3\Lib\site-packages\xgboost\training.py:183: UserWarning: [16:05:29] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


In [1301]:
print(classification_report(tt_y, model5.predict(s_tt_x)))
print(classification_report(tt_y, model6.predict(s_tt_x)))

              precision    recall  f1-score   support

         0.0       0.95      0.66      0.78      1805
         1.0       0.11      0.56      0.19       137

    accuracy                           0.65      1942
   macro avg       0.53      0.61      0.48      1942
weighted avg       0.89      0.65      0.74      1942

              precision    recall  f1-score   support

         0.0       0.93      0.98      0.96      1805
         1.0       0.24      0.08      0.12       137

    accuracy                           0.92      1942
   macro avg       0.59      0.53      0.54      1942
weighted avg       0.88      0.92      0.90      1942



In [1302]:
print(f1_score(tt_y, model1.predict(s_tt_x)))
print(f1_score(tt_y, model2.predict(s_tt_x)))
print(f1_score(tt_y, model3.predict(s_tt_x)))
print(f1_score(tt_y, model4.predict(s_tt_x)))
print(f1_score(tt_y, model5.predict(s_tt_x)))
print(f1_score(tt_y, model6.predict(s_tt_x)))

0.15306122448979592
0.11956521739130435
0.17006802721088435
0.18510158013544017
0.18621523579201935
0.12021857923497267


In [1303]:
grid_model1 = RandomForestClassifier()

grid_model1_p = {
    'n_estimators': [100, 200],
    'max_depth': [None, 10, 20],
    'min_samples_split': [2, 5],
    'min_samples_leaf': [1, 2],
    'max_features': ['sqrt', 'log2']
}

grid_model1 = GridSearchCV(grid_model1, grid_model1_p, cv=3, scoring='f1', n_jobs=-1)
grid_model1.fit(s_tr_x, r_tr_y)

print(grid_model1.best_params_)
print(grid_model1.best_score_)

C:\Users\campus4D044\anaconda3\Lib\site-packages\sklearn\base.py:1473: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


{'max_depth': None, 'max_features': 'sqrt', 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 200}
0.95966213368792


In [1304]:
grid_model2 = GradientBoostingClassifier()

grid_model2_p = {
    'n_estimators': [100, 200],
    'learning_rate': [0.01, 0.1, 0.3],
    'max_depth': [3, 5],
    'subsample': [0.8, 1]
}

grid_model2 = GridSearchCV(grid_model2, grid_model2_p, cv=3, scoring='f1', n_jobs=-1)
grid_model2.fit(s_tr_x, r_tr_y)

print(grid_model2.best_params_)
print(grid_model2.best_score_)

C:\Users\campus4D044\anaconda3\Lib\site-packages\sklearn\preprocessing\_label.py:114: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


{'learning_rate': 0.3, 'max_depth': 5, 'n_estimators': 200, 'subsample': 1}
0.9467000565434219


In [1305]:
grid_model3 = XGBClassifier(use_label_encoder=False, eval_metric='logloss')

grid_model3_p = {
    'n_estimators': [100, 200],
    'max_depth': [3, 5, 7],
    'learning_rate': [0.01, 0.1, 0.3],
    'subsample': [0.8, 1],
    'colsample_bytree': [0.8, 1]
}

grid_model3 = GridSearchCV(grid_model3, grid_model3_p, cv=3, scoring='f1', n_jobs=-1)
grid_model3.fit(s_tr_x, r_tr_y)

print(grid_model3.best_params_)
print(grid_model3.best_score_)

C:\Users\campus4D044\anaconda3\Lib\site-packages\xgboost\training.py:183: UserWarning: [16:07:41] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


{'colsample_bytree': 1, 'learning_rate': 0.1, 'max_depth': 7, 'n_estimators': 200, 'subsample': 1}
0.95265884662551


In [1306]:
grid_model4 = SVC()

grid_model4_p = {
    'C': [0.1, 1, 10],
    'kernel': ['linear', 'rbf'],
    'gamma': ['scale', 'auto', 0.01, 0.001]
}

grid_model4 = GridSearchCV(grid_model4, grid_model4_p, cv=3, scoring='accuracy', n_jobs=-1)
grid_model4.fit(s_tr_x, r_tr_y)

print(grid_model4.best_params_)
print(grid_model4.best_score_)

C:\Users\campus4D044\anaconda3\Lib\site-packages\sklearn\utils\validation.py:1339: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


{'C': 10, 'gamma': 'auto', 'kernel': 'rbf'}
0.9394441416469679


In [1307]:
grid_model1 = RandomForestClassifier(max_depth=None, max_features='sqrt', min_samples_leaf=1, min_samples_split=2, n_estimators=200).fit(s_tr_x, r_tr_y)
grid_model2 = GradientBoostingClassifier(learning_rate=0.3, max_depth=5, n_estimators=200, subsample=1).fit(s_tr_x, r_tr_y)
grid_model3 = XGBClassifier(use_label_encoder=False, eval_metric='logloss', colsample_bytree=1, learning_rate=0.3, max_depth=7, n_estimators=100, subsample=1).fit(s_tr_x, r_tr_y)
grid_model4 = SVC(C=10, gamma='scale', kernel='rbf').fit(s_tr_x, r_tr_y)

C:\Users\campus4D044\anaconda3\Lib\site-packages\sklearn\base.py:1473: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
C:\Users\campus4D044\anaconda3\Lib\site-packages\sklearn\preprocessing\_label.py:114: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\campus4D044\anaconda3\Lib\site-packages\xgboost\training.py:183: UserWarning: [16:08:34] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
C:\Users\campus4D044\anaconda3\Lib\site-packages\sklearn\utils\validation.py:1339: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape

In [1308]:
print(f1_score(tt_y, model1.predict(s_tt_x)))
print(f1_score(tt_y, model2.predict(s_tt_x)))
print(f1_score(tt_y, model3.predict(s_tt_x)))
print(f1_score(tt_y, model4.predict(s_tt_x)))

0.15306122448979592
0.11956521739130435
0.17006802721088435
0.18510158013544017


### 성능 - 얘네로 모델링됨

In [67]:
htn_x = base_merged_r[['T_SEX','T_AGE','T_HEIGHT','T_WEIGHT','T_WAIST','T_SBP','T_DBP','T_SMOKE','T_DRINK']]
dm_x = base_merged_r[['T_GLU0','T_WEIGHT','T_WAIST','T_AGE','T_TG','T_HDL','T_DRINK','T_ALT']]
lip_x = base_merged_r[['T_TCHL','T_LDL','T_HDL','T_TG','T_WEIGHT','T_DRINK']]

htn_y = base_merged_r[['T_HTN']]
dm_y = base_merged_r[['T_DM']]
lip_y = base_merged_r[['T_LIP']]

htn_y = htn_y.replace({1.0: 0, 2.0: 1}).astype(int)
dm_y = dm_y.replace({1.0: 0, 2.0: 1}).astype(int)
lip_y = lip_y.replace({1.0: 0, 2.0: 1}).astype(int)

### HTN 예측

In [68]:
tr_x, tt_x, tr_y, tt_y = train_test_split(htn_x, htn_y, train_size=0.7, random_state=10, stratify=htn_y)
tr_x.shape, tt_x.shape, tr_y.shape, tt_y.shape

((6245, 9), (2677, 9), (6245, 1), (2677, 1))

In [69]:
smote1 = SMOTE(random_state=10)
r_tr_x, r_tr_y = smote1.fit_resample(tr_x, tr_y)

In [70]:
ss1 = StandardScaler().fit(r_tr_x)
htn_scaler = StandardScaler().fit(r_tr_x)
s_tr_x = ss1.transform(r_tr_x)
s_tt_x = ss1.transform(tt_x)

In [71]:
model1 = DecisionTreeClassifier(random_state=10).fit(s_tr_x,r_tr_y)
model2 = RandomForestClassifier(n_estimators=100,max_depth=None,class_weight='balanced',random_state=10).fit(s_tr_x,r_tr_y)
model3 = GradientBoostingClassifier(n_estimators=100,learning_rate=0.1,max_depth=3,random_state=10).fit(s_tr_x,r_tr_y)
model4 = SVC(kernel='rbf', C=1.0, gamma='scale', random_state=10).fit(s_tr_x,r_tr_y)

C:\Users\campus4D044\anaconda3\Lib\site-packages\sklearn\base.py:1473: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
C:\Users\campus4D044\anaconda3\Lib\site-packages\sklearn\preprocessing\_label.py:114: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\campus4D044\anaconda3\Lib\site-packages\sklearn\utils\validation.py:1339: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [72]:
htn_model = SVC(kernel='rbf', C=1.0, gamma='scale', random_state=10).fit(s_tr_x,r_tr_y)

C:\Users\campus4D044\anaconda3\Lib\site-packages\sklearn\utils\validation.py:1339: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [73]:
print(classification_report(tt_y, model1.predict(s_tt_x)))
print()
print(classification_report(tt_y, model2.predict(s_tt_x)))
print()
print(classification_report(tt_y, model3.predict(s_tt_x)))
print()
print(classification_report(tt_y, model4.predict(s_tt_x)))

              precision    recall  f1-score   support

           0       0.83      0.79      0.81      2140
           1       0.31      0.37      0.34       537

    accuracy                           0.71      2677
   macro avg       0.57      0.58      0.58      2677
weighted avg       0.73      0.71      0.72      2677


              precision    recall  f1-score   support

           0       0.84      0.87      0.86      2140
           1       0.41      0.36      0.38       537

    accuracy                           0.77      2677
   macro avg       0.63      0.61      0.62      2677
weighted avg       0.76      0.77      0.76      2677


              precision    recall  f1-score   support

           0       0.87      0.83      0.85      2140
           1       0.43      0.53      0.47       537

    accuracy                           0.77      2677
   macro avg       0.65      0.68      0.66      2677
weighted avg       0.79      0.77      0.77      2677


              pr

In [58]:
model5 = XGBClassifier(n_estimators=100,learning_rate=0.1,max_depth=3,scale_pos_weight=4,random_state=10,use_label_encoder=False,eval_metric='logloss').fit(s_tr_x,r_tr_y)
model6 = XGBClassifier(random_state=10).fit(s_tr_x,r_tr_y)

C:\Users\campus4D044\anaconda3\Lib\site-packages\xgboost\training.py:183: UserWarning: [13:42:06] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


In [59]:
print(classification_report(tt_y, model5.predict(s_tt_x)))
print(classification_report(tt_y, model6.predict(s_tt_x)))

              precision    recall  f1-score   support

         0.0       0.95      0.47      0.63      2140
         1.0       0.30      0.89      0.45       537

    accuracy                           0.56      2677
   macro avg       0.62      0.68      0.54      2677
weighted avg       0.82      0.56      0.59      2677

              precision    recall  f1-score   support

         0.0       0.84      0.89      0.87      2140
         1.0       0.44      0.34      0.39       537

    accuracy                           0.78      2677
   macro avg       0.64      0.62      0.63      2677
weighted avg       0.76      0.78      0.77      2677



In [60]:
print(f1_score(tt_y, model1.predict(s_tt_x)))
print(f1_score(tt_y, model2.predict(s_tt_x)))
print(f1_score(tt_y, model3.predict(s_tt_x)))
print(f1_score(tt_y, model4.predict(s_tt_x)))
print(f1_score(tt_y, model5.predict(s_tt_x)))
print(f1_score(tt_y, model6.predict(s_tt_x)))

0.3549201009251472
0.3988549618320611
0.46755407653910147
0.48433420365535246
0.4468283582089552
0.38662486938349006


In [1318]:
grid_model1 = RandomForestClassifier()

grid_model1_p = {
    'n_estimators': [100, 200],
    'max_depth': [None, 10, 20],
    'min_samples_split': [2, 5],
    'min_samples_leaf': [1, 2],
    'max_features': ['sqrt', 'log2']
}

grid_model1 = GridSearchCV(grid_model1, grid_model1_p, cv=3, scoring='f1', n_jobs=-1)
grid_model1.fit(s_tr_x, r_tr_y)

print(grid_model1.best_params_)
print(grid_model1.best_score_)

C:\Users\campus4D044\anaconda3\Lib\site-packages\sklearn\base.py:1473: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


{'max_depth': None, 'max_features': 'sqrt', 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 200}
0.8717518224494304


In [1319]:
grid_model2 = GradientBoostingClassifier()

grid_model2_p = {
    'n_estimators': [100, 200],
    'learning_rate': [0.01, 0.1, 0.3],
    'max_depth': [3, 5],
    'subsample': [0.8, 1]
}

grid_model2 = GridSearchCV(grid_model2, grid_model2_p, cv=3, scoring='f1', n_jobs=-1)
grid_model2.fit(s_tr_x, r_tr_y)

print(grid_model2.best_params_)
print(grid_model2.best_score_)

C:\Users\campus4D044\anaconda3\Lib\site-packages\sklearn\preprocessing\_label.py:114: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


{'learning_rate': 0.1, 'max_depth': 5, 'n_estimators': 100, 'subsample': 1}
0.8275151476845721


In [1320]:
grid_model3 = XGBClassifier(use_label_encoder=False, eval_metric='logloss')

grid_model3_p = {
    'n_estimators': [100, 200],
    'max_depth': [3, 5, 7],
    'learning_rate': [0.01, 0.1, 0.3],
    'subsample': [0.8, 1],
    'colsample_bytree': [0.8, 1]
}

grid_model3 = GridSearchCV(grid_model3, grid_model3_p, cv=3, scoring='f1', n_jobs=-1)
grid_model3.fit(s_tr_x, r_tr_y)

print(grid_model3.best_params_)
print(grid_model3.best_score_)

{'colsample_bytree': 1, 'learning_rate': 0.1, 'max_depth': 7, 'n_estimators': 100, 'subsample': 1}
0.8385347864284284


C:\Users\campus4D044\anaconda3\Lib\site-packages\xgboost\training.py:183: UserWarning: [16:09:52] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


In [1321]:
grid_model4 = SVC()

grid_model4_p = {
    'C': [0.1, 1, 10],
    'kernel': ['linear', 'rbf'],
    'gamma': ['scale', 'auto', 0.01, 0.001]
}

grid_model4 = GridSearchCV(grid_model4, grid_model4_p, cv=3, scoring='accuracy', n_jobs=-1)
grid_model4.fit(s_tr_x, r_tr_y)

print(grid_model4.best_params_)
print(grid_model4.best_score_)

C:\Users\campus4D044\anaconda3\Lib\site-packages\sklearn\utils\validation.py:1339: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


{'C': 10, 'gamma': 'scale', 'kernel': 'rbf'}
0.7986495242282713


In [1322]:
grid_model1 = RandomForestClassifier(max_depth=20, max_features='log2', min_samples_leaf=1, min_samples_split=2, n_estimators=200).fit(s_tr_x, r_tr_y)
grid_model2 = GradientBoostingClassifier(learning_rate=0.1, max_depth=5, n_estimators=100, subsample=1).fit(s_tr_x, r_tr_y)
grid_model3 = XGBClassifier(use_label_encoder=False, eval_metric='logloss', colsample_bytree=1, learning_rate=0.1, max_depth=7, n_estimators=100, subsample=1).fit(s_tr_x, r_tr_y)
grid_model4 = SVC(C=10, gamma='auto', kernel='rbf').fit(s_tr_x, r_tr_y)

C:\Users\campus4D044\anaconda3\Lib\site-packages\sklearn\base.py:1473: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
C:\Users\campus4D044\anaconda3\Lib\site-packages\sklearn\preprocessing\_label.py:114: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\campus4D044\anaconda3\Lib\site-packages\xgboost\training.py:183: UserWarning: [16:10:23] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
C:\Users\campus4D044\anaconda3\Lib\site-packages\sklearn\utils\validation.py:1339: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape

In [1323]:
print(f1_score(tt_y, grid_model1.predict(s_tt_x)))
print(f1_score(tt_y, grid_model2.predict(s_tt_x)))
print(f1_score(tt_y, grid_model3.predict(s_tt_x)))
print(f1_score(tt_y, grid_model4.predict(s_tt_x)))

0.3523809523809524
0.391653290529695
0.3751987281399046
0.41202672605790647


In [1324]:
best_model = RandomForestClassifier(max_depth=None, max_features='log2', min_samples_leaf=1, min_samples_split=2, n_estimators=200).fit(s_tr_x, r_tr_y)

C:\Users\campus4D044\anaconda3\Lib\site-packages\sklearn\base.py:1473: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


In [1325]:
print(classification_report(tt_y, best_model.predict(s_tt_x)))

              precision    recall  f1-score   support

         0.0       0.87      0.88      0.88      1617
         1.0       0.37      0.36      0.37       325

    accuracy                           0.79      1942
   macro avg       0.62      0.62      0.62      1942
weighted avg       0.79      0.79      0.79      1942



### DM 예측

In [104]:
tr_x, tt_x, tr_y, tt_y = train_test_split(dm_x, dm_y, train_size=0.7, random_state=10, stratify=dm_y)
tr_x.shape, tt_x.shape, tr_y.shape, tt_y.shape

((6245, 8), (2677, 8), (6245, 1), (2677, 1))

In [105]:
smote2 = SMOTE(random_state=10)
r_tr_x, r_tr_y = smote2.fit_resample(tr_x, tr_y)

In [106]:
ss2 = StandardScaler().fit(r_tr_x)
dm_scaler = StandardScaler().fit(r_tr_x)
s_tr_x = ss2.transform(r_tr_x)
s_tt_x = ss2.transform(tt_x)

In [107]:
model1 = DecisionTreeClassifier(random_state=10).fit(s_tr_x,r_tr_y)
model2 = RandomForestClassifier(n_estimators=100,max_depth=None,class_weight='balanced',random_state=10).fit(s_tr_x,r_tr_y)
model3 = GradientBoostingClassifier(n_estimators=100,learning_rate=0.1,max_depth=3,random_state=10).fit(s_tr_x,r_tr_y)
model4 = SVC(kernel='rbf', C=1.0, gamma='scale', random_state=10).fit(s_tr_x,r_tr_y)
model5 = XGBClassifier(random_state=10).fit(s_tr_x,r_tr_y)

C:\Users\campus4D044\anaconda3\Lib\site-packages\sklearn\base.py:1473: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
C:\Users\campus4D044\anaconda3\Lib\site-packages\sklearn\preprocessing\_label.py:114: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\campus4D044\anaconda3\Lib\site-packages\sklearn\utils\validation.py:1339: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [108]:
dm_model = RandomForestClassifier(n_estimators=100,max_depth=None,class_weight='balanced',random_state=10).fit(s_tr_x,r_tr_y)

C:\Users\campus4D044\anaconda3\Lib\site-packages\sklearn\base.py:1473: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


In [109]:
print('Decision Tree Classifier')
print(classification_report(tt_y, model1.predict(s_tt_x)))
print()
print('Random Forest Classifier')
print(classification_report(tt_y, model2.predict(s_tt_x)))
print()
print('Gradient Boosting Classifier')
print(classification_report(tt_y, model3.predict(s_tt_x)))
print()
print('Support Vector Classifier')
print(classification_report(tt_y, model4.predict(s_tt_x)))
print()
print('XG Booster Classifier')
print(classification_report(tt_y, model5.predict(s_tt_x)))

Decision Tree Classifier
              precision    recall  f1-score   support

           0       0.96      0.92      0.94      2492
           1       0.32      0.49      0.39       185

    accuracy                           0.89      2677
   macro avg       0.64      0.71      0.67      2677
weighted avg       0.92      0.89      0.90      2677


Random Forest Classifier
              precision    recall  f1-score   support

           0       0.97      0.97      0.97      2492
           1       0.56      0.61      0.58       185

    accuracy                           0.94      2677
   macro avg       0.77      0.79      0.78      2677
weighted avg       0.94      0.94      0.94      2677


Gradient Boosting Classifier
              precision    recall  f1-score   support

           0       0.98      0.95      0.96      2492
           1       0.50      0.69      0.58       185

    accuracy                           0.93      2677
   macro avg       0.74      0.82      0.77    

In [110]:
print('Decision Tree Classifier')
print(f1_score(tt_y, model1.predict(s_tt_x)))
print()
print('Random Forest Classifier')
print(f1_score(tt_y, model2.predict(s_tt_x)))
print()
print('Gradient Boosting Classifier')
print(f1_score(tt_y, model3.predict(s_tt_x)))
print()
print('Support Vector Classifier')
print(f1_score(tt_y, model4.predict(s_tt_x)))
print()
print('XG Booster Classifier')
print(f1_score(tt_y, model5.predict(s_tt_x)))

Decision Tree Classifier
0.38876889848812096

Random Forest Classifier
0.5833333333333334

Gradient Boosting Classifier
0.5759637188208617

Support Vector Classifier
0.48120300751879697

XG Booster Classifier
0.5525525525525525


In [54]:
model5 = XGBClassifier(n_estimators=100,learning_rate=0.1,max_depth=3,scale_pos_weight=4,random_state=10,use_label_encoder=False,eval_metric='logloss').fit(s_tr_x,r_tr_y)
model6 = XGBClassifier(random_state=10).fit(s_tr_x,r_tr_y)

C:\Users\campus4D044\anaconda3\Lib\site-packages\xgboost\training.py:183: UserWarning: [20:00:10] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


In [55]:
print(classification_report(tt_y, model5.predict(s_tt_x)))
print(classification_report(tt_y, model6.predict(s_tt_x)))

              precision    recall  f1-score   support

           0       0.99      0.83      0.90      2492
           1       0.27      0.85      0.41       185

    accuracy                           0.83      2677
   macro avg       0.63      0.84      0.65      2677
weighted avg       0.94      0.83      0.87      2677

              precision    recall  f1-score   support

           0       0.96      0.98      0.97      2492
           1       0.62      0.50      0.55       185

    accuracy                           0.94      2677
   macro avg       0.79      0.74      0.76      2677
weighted avg       0.94      0.94      0.94      2677



In [53]:
print(f1_score(tt_y, model1.predict(s_tt_x)))
print(f1_score(tt_y, model2.predict(s_tt_x)))
print(f1_score(tt_y, model3.predict(s_tt_x)))
print(f1_score(tt_y, model4.predict(s_tt_x)))
print(f1_score(tt_y, model5.predict(s_tt_x)))
print(f1_score(tt_y, model6.predict(s_tt_x)))

0.38876889848812096
0.5833333333333334
0.5759637188208617
0.48120300751879697
0.4067357512953368
0.5525525525525525


In [841]:
grid_model1 = RandomForestClassifier()

grid_model1_p = {
    'n_estimators': [100, 200],
    'max_depth': [None, 10, 20],
    'min_samples_split': [2, 5],
    'min_samples_leaf': [1, 2],
    'max_features': ['sqrt', 'log2']
}

grid_model1 = GridSearchCV(grid_model1, grid_model1_p, cv=3, scoring='f1', n_jobs=-1)
grid_model1.fit(s_tr_x, r_tr_y)

print(grid_model1.best_params_)
print(grid_model1.best_score_)

C:\Users\campus4D044\anaconda3\Lib\site-packages\sklearn\base.py:1473: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


{'max_depth': None, 'max_features': 'log2', 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 100}
0.9625745937903162


In [842]:
grid_model2 = GradientBoostingClassifier()

grid_model2_p = {
    'n_estimators': [100, 200],
    'learning_rate': [0.01, 0.1, 0.3],
    'max_depth': [3, 5],
    'subsample': [0.8, 1]
}

grid_model2 = GridSearchCV(grid_model2, grid_model2_p, cv=3, scoring='f1', n_jobs=-1)
grid_model2.fit(s_tr_x, r_tr_y)

print(grid_model2.best_params_)
print(grid_model2.best_score_)

C:\Users\campus4D044\anaconda3\Lib\site-packages\sklearn\preprocessing\_label.py:114: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


{'learning_rate': 0.3, 'max_depth': 5, 'n_estimators': 200, 'subsample': 1}
0.958799049684785


In [843]:
grid_model3 = XGBClassifier(use_label_encoder=False, eval_metric='logloss')

grid_model3_p = {
    'n_estimators': [100, 200],
    'max_depth': [3, 5, 7],
    'learning_rate': [0.01, 0.1, 0.3],
    'subsample': [0.8, 1],
    'colsample_bytree': [0.8, 1]
}

grid_model3 = GridSearchCV(grid_model3, grid_model3_p, cv=3, scoring='f1', n_jobs=-1)
grid_model3.fit(s_tr_x, r_tr_y)

print(grid_model3.best_params_)
print(grid_model3.best_score_)

C:\Users\campus4D044\anaconda3\Lib\site-packages\xgboost\training.py:183: UserWarning: [14:08:07] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


{'colsample_bytree': 0.8, 'learning_rate': 0.1, 'max_depth': 7, 'n_estimators': 200, 'subsample': 0.8}
0.9666618717339149


In [844]:
grid_model4 = SVC()

grid_model4_p = {
    'C': [0.1, 1, 10],
    'kernel': ['linear', 'rbf'],
    'gamma': ['scale', 'auto', 0.01, 0.001]
}

grid_model4 = GridSearchCV(grid_model4, grid_model4_p, cv=3, scoring='accuracy', n_jobs=-1)
grid_model4.fit(s_tr_x, r_tr_y)

print(grid_model4.best_params_)
print(grid_model4.best_score_)

C:\Users\campus4D044\anaconda3\Lib\site-packages\sklearn\utils\validation.py:1339: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


{'C': 10, 'gamma': 'auto', 'kernel': 'rbf'}
0.9521928782011063


In [845]:
grid_model1 = RandomForestClassifier(max_depth=20, max_features='log2', min_samples_leaf=1, min_samples_split=2, n_estimators=200).fit(s_tr_x, r_tr_y)
grid_model2 = GradientBoostingClassifier(learning_rate=0.1, max_depth=5, n_estimators=100, subsample=1).fit(s_tr_x, r_tr_y)
grid_model3 = XGBClassifier(use_label_encoder=False, eval_metric='logloss', colsample_bytree=1, learning_rate=0.1, max_depth=7, n_estimators=100, subsample=1).fit(s_tr_x, r_tr_y)
grid_model4 = SVC(C=10, gamma='auto', kernel='rbf').fit(s_tr_x, r_tr_y)

C:\Users\campus4D044\anaconda3\Lib\site-packages\sklearn\base.py:1473: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
C:\Users\campus4D044\anaconda3\Lib\site-packages\sklearn\preprocessing\_label.py:114: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\campus4D044\anaconda3\Lib\site-packages\xgboost\training.py:183: UserWarning: [14:09:04] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
C:\Users\campus4D044\anaconda3\Lib\site-packages\sklearn\utils\validation.py:1339: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape

In [846]:
print(f1_score(tt_y, grid_model1.predict(s_tt_x)))
print(f1_score(tt_y, grid_model2.predict(s_tt_x)))
print(f1_score(tt_y, grid_model3.predict(s_tt_x)))
print(f1_score(tt_y, grid_model4.predict(s_tt_x)))

0.38589211618257263
0.5799457994579946
0.5545454545454546
0.5115303983228512


In [847]:
best_model = RandomForestClassifier(max_depth=None, max_features='sqrt', min_samples_leaf=1, min_samples_split=2, n_estimators=200).fit(s_tr_x, r_tr_y)

C:\Users\campus4D044\anaconda3\Lib\site-packages\sklearn\base.py:1473: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


In [848]:
print(classification_report(tt_y, best_model.predict(s_tt_x)))

              precision    recall  f1-score   support

         0.0       0.97      0.97      0.97      2492
         1.0       0.58      0.58      0.58       185

    accuracy                           0.94      2677
   macro avg       0.78      0.77      0.77      2677
weighted avg       0.94      0.94      0.94      2677



### LIP 예측

In [74]:
tr_x, tt_x, tr_y, tt_y = train_test_split(lip_x, lip_y, train_size=0.7, random_state=10, stratify=lip_y)
tr_x.shape, tt_x.shape, tr_y.shape, tt_y.shape

((6245, 6), (2677, 6), (6245, 1), (2677, 1))

In [75]:
smote3 = SMOTE(random_state=10)
r_tr_x, r_tr_y = smote3.fit_resample(tr_x, tr_y)

In [76]:
ss3 = StandardScaler().fit(r_tr_x)
lip_scaler = StandardScaler().fit(r_tr_x)
s_tr_x = ss3.transform(r_tr_x)
s_tt_x = ss3.transform(tt_x)

In [77]:
model1 = DecisionTreeClassifier(random_state=10).fit(s_tr_x,r_tr_y)
model2 = RandomForestClassifier(n_estimators=100,max_depth=None,class_weight='balanced',random_state=10).fit(s_tr_x,r_tr_y)
model3 = GradientBoostingClassifier(n_estimators=100,learning_rate=0.1,max_depth=3,random_state=10).fit(s_tr_x,r_tr_y)
model4 = SVC(kernel='rbf', C=1.0, gamma='scale', random_state=10).fit(s_tr_x,r_tr_y)

lip_model = SVC(kernel='rbf', C=1.0, gamma='scale', random_state=10).fit(s_tr_x,r_tr_y)

C:\Users\campus4D044\anaconda3\Lib\site-packages\sklearn\base.py:1473: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
C:\Users\campus4D044\anaconda3\Lib\site-packages\sklearn\preprocessing\_label.py:114: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\campus4D044\anaconda3\Lib\site-packages\sklearn\utils\validation.py:1339: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\campus4D044\anaconda3\Lib\site-packages\sklearn\utils\validation.py:1339: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change 

In [47]:
print(classification_report(tt_y, model1.predict(s_tt_x)))
print()
print(classification_report(tt_y, model2.predict(s_tt_x)))
print()
print(classification_report(tt_y, model3.predict(s_tt_x)))
print()
print(classification_report(tt_y, model4.predict(s_tt_x)))

              precision    recall  f1-score   support

         0.0       0.92      0.86      0.89      2464
         1.0       0.10      0.18      0.13       213

    accuracy                           0.81      2677
   macro avg       0.51      0.52      0.51      2677
weighted avg       0.86      0.81      0.83      2677


              precision    recall  f1-score   support

         0.0       0.92      0.95      0.93      2464
         1.0       0.12      0.09      0.10       213

    accuracy                           0.88      2677
   macro avg       0.52      0.52      0.52      2677
weighted avg       0.86      0.88      0.87      2677


              precision    recall  f1-score   support

         0.0       0.93      0.93      0.93      2464
         1.0       0.14      0.14      0.14       213

    accuracy                           0.87      2677
   macro avg       0.54      0.53      0.53      2677
weighted avg       0.86      0.87      0.87      2677


              pr

In [48]:
model5 = XGBClassifier(n_estimators=100,learning_rate=0.1,max_depth=3,scale_pos_weight=4,random_state=10,use_label_encoder=False,eval_metric='logloss').fit(s_tr_x,r_tr_y)
model6 = XGBClassifier(random_state=10).fit(s_tr_x,r_tr_y)

C:\Users\campus4D044\anaconda3\Lib\site-packages\xgboost\training.py:183: UserWarning: [13:41:34] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


In [49]:
print(classification_report(tt_y, model5.predict(s_tt_x)))
print(classification_report(tt_y, model6.predict(s_tt_x)))

              precision    recall  f1-score   support

         0.0       0.95      0.40      0.56      2464
         1.0       0.10      0.74      0.17       213

    accuracy                           0.42      2677
   macro avg       0.52      0.57      0.36      2677
weighted avg       0.88      0.42      0.53      2677

              precision    recall  f1-score   support

         0.0       0.92      0.97      0.95      2464
         1.0       0.13      0.05      0.07       213

    accuracy                           0.90      2677
   macro avg       0.53      0.51      0.51      2677
weighted avg       0.86      0.90      0.88      2677



In [50]:
print(f1_score(tt_y, model1.predict(s_tt_x)))
print(f1_score(tt_y, model2.predict(s_tt_x)))
print(f1_score(tt_y, model3.predict(s_tt_x)))
print(f1_score(tt_y, model4.predict(s_tt_x)))
print(f1_score(tt_y, model5.predict(s_tt_x)))
print(f1_score(tt_y, model6.predict(s_tt_x)))

0.12751677852348994
0.10382513661202186
0.14043583535108958
0.1913499344692005
0.16908992999461497
0.07407407407407407


In [816]:
grid_model1 = RandomForestClassifier()

grid_model1_p = {
    'n_estimators': [100, 200],
    'max_depth': [None, 10, 20],
    'min_samples_split': [2, 5],
    'min_samples_leaf': [1, 2],
    'max_features': ['sqrt', 'log2']
}

grid_model1 = GridSearchCV(grid_model1, grid_model1_p, cv=3, scoring='f1', n_jobs=-1)
grid_model1.fit(s_tr_x, r_tr_y)

print(grid_model1.best_params_)
print(grid_model1.best_score_)

C:\Users\campus4D044\anaconda3\Lib\site-packages\sklearn\base.py:1473: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


{'max_depth': None, 'max_features': 'sqrt', 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 200}
0.8487071393373414


In [817]:
grid_model2 = GradientBoostingClassifier()

grid_model2_p = {
    'n_estimators': [100, 200],
    'learning_rate': [0.01, 0.1, 0.3],
    'max_depth': [3, 5],
    'subsample': [0.8, 1]
}

grid_model2 = GridSearchCV(grid_model2, grid_model2_p, cv=3, scoring='f1', n_jobs=-1)
grid_model2.fit(s_tr_x, r_tr_y)

print(grid_model2.best_params_)
print(grid_model2.best_score_)

C:\Users\campus4D044\anaconda3\Lib\site-packages\sklearn\preprocessing\_label.py:114: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


{'learning_rate': 0.1, 'max_depth': 5, 'n_estimators': 100, 'subsample': 1}
0.7709312876420733


In [818]:
grid_model3 = XGBClassifier(use_label_encoder=False, eval_metric='logloss')

grid_model3_p = {
    'n_estimators': [100, 200],
    'max_depth': [3, 5, 7],
    'learning_rate': [0.01, 0.1, 0.3],
    'subsample': [0.8, 1],
    'colsample_bytree': [0.8, 1]
}

grid_model3 = GridSearchCV(grid_model3, grid_model3_p, cv=3, scoring='f1', n_jobs=-1)
grid_model3.fit(s_tr_x, r_tr_y)

print(grid_model3.best_params_)
print(grid_model3.best_score_)

{'colsample_bytree': 1, 'learning_rate': 0.1, 'max_depth': 7, 'n_estimators': 100, 'subsample': 0.8}
0.7866014122425504


C:\Users\campus4D044\anaconda3\Lib\site-packages\xgboost\training.py:183: UserWarning: [14:01:39] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


In [819]:
grid_model4 = SVC()

grid_model4_p = {
    'C': [0.1, 1, 10],
    'kernel': ['linear', 'rbf'],
    'gamma': ['scale', 'auto', 0.01, 0.001]
}

grid_model4 = GridSearchCV(grid_model4, grid_model4_p, cv=3, scoring='f1', n_jobs=-1)
grid_model4.fit(s_tr_x, r_tr_y)

print(grid_model4.best_params_)
print(grid_model4.best_score_)

C:\Users\campus4D044\anaconda3\Lib\site-packages\sklearn\utils\validation.py:1339: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


{'C': 10, 'gamma': 'auto', 'kernel': 'rbf'}
0.8134035429129723


In [820]:
grid_model1 = RandomForestClassifier(max_depth=20, max_features='log2', min_samples_leaf=1, min_samples_split=2, n_estimators=200).fit(s_tr_x, r_tr_y)
grid_model2 = GradientBoostingClassifier(learning_rate=0.1, max_depth=5, n_estimators=100, subsample=1).fit(s_tr_x, r_tr_y)
grid_model3 = XGBClassifier(use_label_encoder=False, eval_metric='logloss', colsample_bytree=1, learning_rate=0.1, max_depth=7, n_estimators=100, subsample=1).fit(s_tr_x, r_tr_y)
grid_model4 = SVC(C=10, gamma='auto', kernel='rbf').fit(s_tr_x, r_tr_y)

C:\Users\campus4D044\anaconda3\Lib\site-packages\sklearn\base.py:1473: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
C:\Users\campus4D044\anaconda3\Lib\site-packages\sklearn\preprocessing\_label.py:114: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\campus4D044\anaconda3\Lib\site-packages\xgboost\training.py:183: UserWarning: [14:02:41] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
C:\Users\campus4D044\anaconda3\Lib\site-packages\sklearn\utils\validation.py:1339: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape

In [821]:
print(f1_score(tt_y, grid_model1.predict(s_tt_x)))
print(f1_score(tt_y, grid_model2.predict(s_tt_x)))
print(f1_score(tt_y, grid_model3.predict(s_tt_x)))
print(f1_score(tt_y, grid_model4.predict(s_tt_x)))

0.3778656126482213
0.438512869399428
0.48582995951417
0.5104311543810849


In [822]:
best_model = RandomForestClassifier(max_depth=None, max_features='log2', min_samples_leaf=1, min_samples_split=2, n_estimators=200).fit(s_tr_x, r_tr_y)

C:\Users\campus4D044\anaconda3\Lib\site-packages\sklearn\base.py:1473: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


In [823]:
print(classification_report(tt_y, best_model.predict(s_tt_x)))

              precision    recall  f1-score   support

         0.0       0.86      0.87      0.86      2140
         1.0       0.46      0.44      0.45       537

    accuracy                           0.78      2677
   macro avg       0.66      0.66      0.66      2677
weighted avg       0.78      0.78      0.78      2677



### 변수 선택

In [172]:
htn_x = base_merged_x[['T_AGE','T_HEIGHT','T_WEIGHT','T_WAIST','T_SBP','T_DBP','T_LDL','T_GLU0','T_CREATINE','T_DRINK']]
dm_x = base_merged_x[['T_SEX','T_AGE','T_HEIGHT','T_WAIST','T_SBP','T_DBP','T_GLU0','T_TCHL','T_TG','T_HDL','T_LDL','T_CREATINE','T_ALT','T_SMOKE','T_DRINK']]
lip_x = base_merged_x[['T_SEX','T_AGE','T_HEIGHT','T_WEIGHT','T_SBP','T_TCHL','T_TG','T_HDL','T_LDL','T_AST','T_ALT','T_SMOKE','T_DRINK']]

In [55]:
tr_x1, tt_x1, tr_y1, tt_y1 = train_test_split(htn_x, base_merged_htn, train_size=0.7, random_state=10, stratify=base_merged_htn)
smote1 = SMOTE(random_state=10)
r_tr_x1, r_tr_y1 = smote1.fit_resample(tr_x1, tr_y1)
ss1 = StandardScaler().fit(r_tr_x1)
s_tr_x1 = ss1.transform(r_tr_x1)
s_tt_x1 = ss1.transform(tt_x1)

tr_x2, tt_x2, tr_y2, tt_y2 = train_test_split(dm_x, base_merged_dm, train_size=0.7, random_state=10, stratify=base_merged_dm)
smote2 = SMOTE(random_state=10)
r_tr_x2, r_tr_y2 = smote2.fit_resample(tr_x2, tr_y2)
ss2 = StandardScaler().fit(r_tr_x2)
s_tr_x2 = ss2.transform(r_tr_x2)
s_tt_x2 = ss2.transform(tt_x2)

tr_x3, tt_x3, tr_y3, tt_y3 = train_test_split(lip_x, base_merged_lip, train_size=0.7, random_state=10, stratify=base_merged_lip)
smote3 = SMOTE(random_state=10)
r_tr_x3, r_tr_y3 = smote3.fit_resample(tr_x3, tr_y3)
ss3 = StandardScaler().fit(r_tr_x3)
s_tr_x3 = ss3.transform(r_tr_x3)
s_tt_x3 = ss3.transform(tt_x3)

In [56]:
model1 = SVC(random_state=10).fit(s_tr_x1,r_tr_y1)
model2 = RandomForestClassifier(random_state=10).fit(s_tr_x2,r_tr_y2)
model3 = SVC(random_state=10).fit(s_tr_x3,r_tr_y3)

C:\Users\campus4D044\anaconda3\Lib\site-packages\sklearn\utils\validation.py:1339: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\campus4D044\anaconda3\Lib\site-packages\sklearn\base.py:1473: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
C:\Users\campus4D044\anaconda3\Lib\site-packages\sklearn\utils\validation.py:1339: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [58]:
print(f1_score(tt_y1,model1.predict(s_tt_x1)))
print(f1_score(tt_y2,model2.predict(s_tt_x2)))
print(f1_score(tt_y3,model3.predict(s_tt_x3)))

0.5130674002751031
0.5951742627345844
0.2282208588957055


HTN
htn_x = base_merged_x[['T_AGE','T_HEIGHT','T_WEIGHT','T_WAIST','T_SBP','T_DBP','T_LDL','T_GLU0','T_CREATINE','T_DRINK']]
0.5130674002751031 (SVC)
htn_x = base_merged_x[['T_AGE','T_HEIGHT','T_WEIGHT','T_WAIST','T_SBP','T_LDL','T_GLU0','T_CREATINE','T_DRINK']]
0.51004851004851 (SVC)
htn_x = base_merged_x[['T_AGE','T_HEIGHT','T_WEIGHT','T_WAIST','T_SBP','T_DBP','T_LDL','T_GLU0','T_TCHL','T_CREATINE','T_DRINK']]
0.5117565698478561 (SVC)
htn_x = base_merged_x[['T_AGE','T_HEIGHT','T_WEIGHT','T_WAIST','T_SBP','T_DBP','T_LDL','T_GLU0','T_TCHL','T_CREATINE','T_ALT','T_DRINK']]
0.5104311543810849 (SVC)

DM
dm_x = base_merged_x[['T_SEX','T_AGE','T_HEIGHT','T_WEIGHT','T_WAIST','T_SBP','T_DBP','T_GLU0','T_TCHL','T_TG','T_HDL','T_LDL','T_CREATINE','T_AST','T_ALT','T_SMOKE','T_DRINK']]
0.5844504021447721 (RandomForestClassifier)
dm_x = base_merged_x[['T_SEX','T_AGE','T_HEIGHT','T_WAIST','T_SBP','T_DBP','T_GLU0','T_TCHL','T_TG','T_HDL','T_LDL','T_CREATINE','T_AST','T_ALT','T_SMOKE','T_DRINK']]
0.5921450151057401 (XGBClassifier)
dm_x = base_merged_x[['T_SEX','T_AGE','T_HEIGHT','T_WAIST','T_SBP','T_DBP','T_GLU0','T_TCHL','T_TG','T_HDL','T_LDL','T_CREATINE','T_ALT','T_SMOKE','T_DRINK']]
0.6112600536193029 (RandomForestClassifier)

LIP
lip_x = base_merged_x[['T_SEX','T_AGE','T_HEIGHT','T_WEIGHT','T_WAIST','T_SBP','T_DBP','T_GLU0','T_TCHL','T_TG','T_HDL','T_LDL','T_CREATINE','T_AST','T_ALT','T_SMOKE','T_DRINK']]
0.222509702457956 (SVC)
lip_x = base_merged_x[['T_SEX','T_AGE','T_HEIGHT','T_WEIGHT','T_SBP','T_DBP','T_GLU0','T_TCHL','T_TG','T_HDL','T_LDL','T_CREATINE','T_AST','T_ALT','T_SMOKE','T_DRINK']]
0.22280471821756226 (SVC)
lip_x = base_merged_x[['T_SEX','T_AGE','T_HEIGHT','T_WEIGHT','T_SBP','T_GLU0','T_TCHL','T_TG','T_HDL','T_LDL','T_CREATINE','T_AST','T_ALT','T_SMOKE','T_DRINK']]
0.22422680412371135 (SVC)
lip_x = base_merged_x[['T_SEX','T_AGE','T_HEIGHT','T_WEIGHT','T_SBP','T_TCHL','T_TG','T_HDL','T_LDL','T_AST','T_ALT','T_SMOKE','T_DRINK']]
0.2282208588957055 (SVC)

### 성능 향상

In [165]:
base_merged_c.columns

Index(['T_ID', 'T_SEX', 'T_AGE', 'T_HTN', 'T_DM', 'T_LIP', 'T_FMFHT1',
       'T_FMFHT2', 'T_FMFDM1', 'T_FMFDM2', 'T_DRINK', 'T_SMOKE', 'T_PSM',
       'T_EXER', 'T_SBP', 'T_DBP', 'T_PULSE', 'T_WAIST', 'T_HIP', 'T_HEIGHT',
       'T_WEIGHT', 'T_BMI', 'T_GLU0', 'T_CREATINE', 'T_AST', 'T_ALT', 'T_TCHL',
       'T_HDL', 'T_LDL', 'T_TG'],
      dtype='object')

In [196]:
dm_x = base_merged_c[['T_SEX','T_AGE','T_HEIGHT','T_WAIST','T_SBP','T_DBP','T_GLU0','T_TCHL','T_TG','T_HDL','T_LDL','T_CREATINE','T_AST','T_ALT','T_SMOKE','T_DRINK']]

In [197]:
tr_x, tt_x, tr_y, tt_y = train_test_split(dm_x, base_merged_dm, train_size=0.7, random_state=10, stratify=base_merged_dm)
smote1 = SMOTE(random_state=10)
r_tr_x, r_tr_y = smote1.fit_resample(tr_x, tr_y)
mms1 = MinMaxScaler().fit(r_tr_x)
s_tr_x = mms1.transform(r_tr_x)
s_tt_x = mms1.transform(tt_x)

In [198]:
grid_model1 = RandomForestClassifier(max_depth=None,criterion='gini',max_features='log2',min_samples_leaf=1,min_samples_split=2,n_estimators=100,class_weight='balanced',random_state=59).fit(s_tr_x, r_tr_y)

C:\Users\campus4D044\anaconda3\Lib\site-packages\sklearn\base.py:1473: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


In [199]:
print('Random Forest Classifier')
print(classification_report(tt_y, grid_model1.predict(s_tt_x)))

Random Forest Classifier
              precision    recall  f1-score   support

         0.0       0.97      0.98      0.98      1883
         1.0       0.18      0.12      0.14        59

    accuracy                           0.96      1942
   macro avg       0.58      0.55      0.56      1942
weighted avg       0.95      0.96      0.95      1942



In [200]:
print('Random Forest Classifier')
print(classification_report(tt_y, grid_model1.predict(s_tt_x)))
print()
print('f1-score')
print(f1_score(tt_y, grid_model1.predict(s_tt_x)))

Random Forest Classifier
              precision    recall  f1-score   support

         0.0       0.97      0.98      0.98      1883
         1.0       0.18      0.12      0.14        59

    accuracy                           0.96      1942
   macro avg       0.58      0.55      0.56      1942
weighted avg       0.95      0.96      0.95      1942


f1-score
0.14432989690721648


In [126]:
grid_model2 =RandomForestClassifier(random_state=59).fit(s_tr_x,r_tr_y)

C:\Users\campus4D044\anaconda3\Lib\site-packages\sklearn\base.py:1473: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


In [127]:
print(f1_score(tt_y, grid_model2.predict(s_tt_x)))

0.14432989690721648


### 적용

In [80]:
medical = pd.read_csv('./data/국민건강보험공단_건강검진정보_2023.csv',encoding='cp949')

In [60]:
medical.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000000 entries, 0 to 999999
Data columns (total 33 columns):
 #   Column         Non-Null Count    Dtype  
---  ------         --------------    -----  
 0   기준년도           1000000 non-null  int64  
 1   가입자일련번호        1000000 non-null  int64  
 2   시도코드           1000000 non-null  int64  
 3   성별코드           1000000 non-null  int64  
 4   연령대코드(5세단위)    1000000 non-null  int64  
 5   신장(5cm단위)      1000000 non-null  int64  
 6   체중(5kg단위)      1000000 non-null  int64  
 7   허리둘레           999589 non-null   float64
 8   시력(좌)          999816 non-null   float64
 9   시력(우)          999823 non-null   float64
 10  청력(좌)          999863 non-null   float64
 11  청력(우)          999862 non-null   float64
 12  수축기혈압          994253 non-null   float64
 13  이완기혈압          994253 non-null   float64
 14  식전혈당(공복혈당)     994186 non-null   float64
 15  총콜레스테롤         338606 non-null   float64
 16  트리글리세라이드       338606 non-null   float64
 17  HDL콜레스테롤 

In [82]:
medical_c = medical[['성별코드','연령대코드(5세단위)','신장(5cm단위)','체중(5kg단위)','허리둘레','수축기혈압','이완기혈압','식전혈당(공복혈당)','총콜레스테롤','트리글리세라이드','HDL콜레스테롤','LDL콜레스테롤','혈청크레아티닌','혈청지오티(AST)','혈청지피티(ALT)','흡연상태','음주여부']]

In [83]:
medical_r = medical_c.dropna()

In [84]:
base_merged_c[['T_ID','T_HTN','T_DM','T_LIP','T_SEX','T_AGE','T_HEIGHT','T_WEIGHT','T_WAIST','T_SBP','T_DBP','T_GLU0','T_TCHL','T_TG','T_HDL','T_LDL','T_CREATINE','T_AST','T_ALT','T_SMOKE','T_DRINK']]

T_ID  T_HTN  T_DM  T_LIP  T_SEX  T_AGE  T_HEIGHT  T_WEIGHT  \
0     K_BASE_00001    1.0   1.0    2.0      2     42     164.0      64.0   
1     K_BASE_00002    2.0   1.0    1.0      2     58     156.0      64.0   
2     K_BASE_00003    2.0   1.0    2.0      2     60     154.0      54.0   
3     K_BASE_00004    2.0   1.0    1.0      2     73     143.0      55.0   
4     K_BASE_00005    1.0   1.0    1.0      2     59     150.0      60.0   
...            ...    ...   ...    ...    ...    ...       ...       ...   
9994  K_BASE_09995    1.0   1.0    1.0      1     44     167.0      62.0   
9996  K_BASE_09997    2.0   2.0    1.0      1     58     168.0      70.0   
9997  K_BASE_09998    1.0   1.0    1.0      1     57     168.0      64.0   
9998  K_BASE_09999    1.0   1.0    1.0      2     43     162.0      62.0   
9999  K_BASE_10000    1.0   1.0    1.0      2     43     148.0      61.0   

      T_WAIST  T_SBP  ...  T_GLU0  T_TCHL   T_TG  T_HDL  T_LDL  T_CREATINE  \
0        82.0  115.0  ...    68.0   172.0   38.0   62.0  102.0        0.60   
1        89.0  143.0  ...    86.0   197.0   79.0   62.0  119.0        0.60   
2        78.0  120.0  ...    86.0   156.0  112.0   46.0   88.0        0.80   
3        86.0  104.0  ...   108.0   170.0  109.0   39.0  109.0        0.76   
4        86.0  125.0  ...    91.0   178.0   52.0   65.0  103.0        0.68   
...       ...    ...  ...     ...     ...    ...    ...    ...         ...   
9994     76.0  103.0  ...   103.0   189.0   63.0   57.0  119.0        0.80   
9996     92.0  130.0  ...   120.0   159.0   93.0   39.0  101.0        0.70   
9997     83.0  100.0  ...    86.0   188.0  389.0   34.0   76.0        1.40   
9998     81.0  120.0  ...   104.0   200.0   54.0   63.0  126.0        0.80   
9999     81.0  134.0  ...    69.0   205.0  142.0   38.0  139.0        0.60   

      T_AST  T_ALT  T_SMOKE  T_DRINK  
0      20.0   12.0      1.0      1.0  
1      17.0   13.0      1.0      2.0  
2      39.0   53.0      1.0      1.0  
3      16.0   28.0      1.0      1.0  
4      18.0   13.0      1.0      1.0  
...     ...    ...      ...      ...  
9994   25.0   25.0      2.0      3.0  
9996   19.0   16.0      2.0      2.0  
9997   22.0   28.0      3.0      3.0  
9998   37.0   46.0      1.0      3.0  
9999   19.0   17.0      1.0      3.0  

[8922 rows x 21 columns]

In [85]:
medical_r['T_SEX'] = medical_r['성별코드'].replace({1: 0, 2: 1})
medical_r['T_AGE'] = medical_r['연령대코드(5세단위)']*5-3
medical_r['T_HEIGHT'] = medical_r['신장(5cm단위)']
medical_r['T_WEIGHT'] = medical_r['체중(5kg단위)']
medical_r['T_WAIST'] = medical_r['허리둘레']
medical_r['T_SBP'] = medical_r['수축기혈압']
medical_r['T_DBP'] = medical_r['이완기혈압']
medical_r['T_GLU0'] = medical_r['식전혈당(공복혈당)']
medical_r['T_TCHL'] = medical_r['총콜레스테롤']
medical_r['T_TG'] = medical_r['트리글리세라이드']
medical_r['T_HDL'] = medical_r['HDL콜레스테롤']
medical_r['T_LDL'] = medical_r['LDL콜레스테롤']
medical_r['T_CREATINE'] = medical_r['혈청크레아티닌']
medical_r['T_AST'] = medical_r['혈청지오티(AST)']
medical_r['T_ALT'] = medical_r['혈청지피티(ALT)']
medical_r['T_SMOKE'] = medical_r['흡연상태'].replace({0: 1, 1: 2})
medical_r['T_DRINK'] = medical_r['음주여부'].replace({0: 0, 1: 3})

C:\Users\campus4D044\AppData\Local\Temp\ipykernel_14928\3398388661.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  medical_r['T_SEX'] = medical_r['성별코드'].replace({1: 0, 2: 1})
C:\Users\campus4D044\AppData\Local\Temp\ipykernel_14928\3398388661.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  medical_r['T_AGE'] = medical_r['연령대코드(5세단위)']*5-3
C:\Users\campus4D044\AppData\Local\Temp\ipykernel_14928\3398388661.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

In [86]:
medical_r = medical_r[['T_SEX','T_AGE','T_HEIGHT','T_WEIGHT','T_WAIST','T_SBP','T_DBP','T_GLU0','T_TCHL','T_TG','T_HDL','T_LDL','T_CREATINE','T_AST','T_ALT','T_SMOKE','T_DRINK']]

In [157]:
htn_x.columns, dm_x.columns, lip_x.columns

(Index(['T_AGE', 'T_AGE', 'T_HEIGHT', 'T_WEIGHT', 'T_WAIST', 'T_SBP', 'T_DBP',
        'T_SMOKE', 'T_DRINK'],
       dtype='object'),
 Index(['T_GLU0', 'T_WEIGHT', 'T_WAIST', 'T_AGE', 'T_TG', 'T_HDL', 'T_DRINK',
        'T_ALT'],
       dtype='object'),
 Index(['T_TCHL', 'T_LDL', 'T_HDL', 'T_TG', 'T_WEIGHT', 'T_DRINK'], dtype='object'))

In [87]:
medical_htn = medical_r[htn_x.columns]
medical_dm = medical_r[dm_x.columns]
medical_lip = medical_r[lip_x.columns]

In [88]:
s_m_htn = htn_scaler.transform(medical_htn)
s_m_dm = dm_scaler.transform(medical_dm)
s_m_lip = lip_scaler.transform(medical_lip)

In [132]:
medical_htn['PREDICT'] = htn_model.predict(s_m_htn)
medical_dm['PREDICT'] = dm_model.predict(s_m_dm)
medical_lip['PREDICT'] = lip_model.predict(s_m_lip)

C:\Users\campus4D044\AppData\Local\Temp\ipykernel_14928\4259373476.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  medical_htn['PREDICT'] = htn_model.predict(s_m_htn)
C:\Users\campus4D044\AppData\Local\Temp\ipykernel_14928\4259373476.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  medical_dm['PREDICT'] = dm_model.predict(s_m_dm)


KeyboardInterrupt: 

In [91]:
medical_htn['htn_y'] = np.where((medical_htn['T_SBP'] >= 140) | (medical_htn['T_DBP'] >= 90), 1, 0)
medical_dm['dm_y'] = np.where(medical_dm['T_GLU0'] >= 126, 1, 0)
medical_lip['lip_y'] = np.where((medical_lip['T_TCHL'] >= 240) | (medical_lip['T_TG'] >= 200) | (medical_lip['T_LDL'] >= 160), 1, 0)

C:\Users\campus4D044\AppData\Local\Temp\ipykernel_14928\4138870531.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  medical_htn['htn_y'] = np.where((medical_htn['T_SBP'] >= 140) | (medical_htn['T_DBP'] >= 90), 1, 0)
C:\Users\campus4D044\AppData\Local\Temp\ipykernel_14928\4138870531.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  medical_dm['dm_y'] = np.where(medical_dm['T_GLU0'] >= 126, 1, 0)
C:\Users\campus4D044\AppData\Local\Temp\ipykernel_14928\4138870531.py:3: SettingWithCopyWarning: 
A value is 

In [130]:
htn_y

T_HTN
0         0
1         1
2         1
3         1
4         0
...     ...
9994      0
9996      1
9997      0
9998      0
9999      0

[8922 rows x 1 columns]

In [ ]:
medical_dm

In [100]:
medical_lip.columns

Index(['T_TCHL', 'T_LDL', 'T_HDL', 'T_TG', 'T_WEIGHT', 'T_DRINK', 'PREDICT',
       'lip_y'],
      dtype='object')

In [95]:
print(f1_score(medical_htn['PREDICT'],medical_htn['htn_y']))
print(f1_score(medical_dm['PREDICT'],medical_dm['dm_y']))
print(f1_score(medical_lip['PREDICT'],medical_lip['lip_y']))

0.3958233820214362
0.6947610266332114
0.3761158108375092


In [133]:
dm_scaler

StandardScaler()

In [129]:
pickle.dump(dm_model,open('박승_당뇨_진단.p','wb'))

In [134]:
pickle.dump(dm_scaler,open('diabetes2_scaler.pkl','wb'))

In [61]:
import pickle
pickle.dump(model4,open('박승_고혈압_진단.p','wb'))

In [42]:
pickle.dump(model2,open('박승_당뇨_진단.p','wb'))

In [52]:
pickle.dump(model4,open('박승_고지혈증_진단.p','wb'))

In [97]:
clustered = pd.read_csv('./data/cluster_result_not_cholesterol.csv',encoding='utf-8')

In [173]:
clustered.columns

Index(['Unnamed: 0', '성별', '연령대', '허리둘레', '수축기혈압', '이완기혈압', '식전혈당(공복혈당)',
       '혈색소', '요단백', '혈청크레아티닌', '혈청지오티(AST)', '혈청지피티(ALT)', '감마지티피', '흡연상태',
       '음주여부', 'BMI구분', '청력', '평균시력', 'new_연령대', 'cluster'],
      dtype='object')

In [175]:
clustered_c = clustered[clustered['cluster'] ==2]
clustered_c = clustered_c[['성별','연령대','허리둘레','수축기혈압','이완기혈압','식전혈당(공복혈당)','혈색소','요단백','혈청크레아티닌','혈청지오티(AST)','혈청지피티(ALT)','감마지티피','흡연상태','음주여부']]

In [176]:
medical_a = medical[(medical['성별코드']==1) & (medical['연령대코드(5세단위)'].isin([5,6,7,8]))]
medical_a = medical[['가입자일련번호','성별코드','연령대코드(5세단위)','허리둘레','수축기혈압','이완기혈압','식전혈당(공복혈당)','혈색소','요단백','혈청크레아티닌','혈청지오티(AST)','혈청지피티(ALT)','감마지티피','흡연상태','음주여부','신장(5cm단위)','체중(5kg단위)']]
medical_m = pd.merge(clustered_c, medical_a, how='left', on=['허리둘레','수축기혈압','이완기혈압','식전혈당(공복혈당)','혈색소','요단백','혈청크레아티닌','혈청지오티(AST)','혈청지피티(ALT)','감마지티피','흡연상태','음주여부'])

In [177]:
medical_m = medical_m[['성별','연령대','허리둘레','수축기혈압', '이완기혈압','식전혈당(공복혈당)','혈색소','요단백','혈청크레아티닌','혈청지오티(AST)','혈청지피티(ALT)','감마지티피','흡연상태','음주여부','BMI구분','cluster','가입자일련번호','신장(5cm단위)', '체중(5kg단위)']]

KeyError: "['BMI구분', 'cluster'] not in index"

In [187]:
medical_m['T_SEX'] = medical_m['성별'].replace({1: 0, 2: 1})
medical_m['T_AGE'] = medical_m['연령대']*5-3
medical_m['T_HEIGHT'] = medical_m['신장(5cm단위)']
medical_m['T_WEIGHT'] = medical_m['체중(5kg단위)']
medical_m['T_WAIST'] = medical_m['허리둘레']
medical_m['T_SBP'] = medical_m['수축기혈압']
medical_m['T_DBP'] = medical_m['이완기혈압']
medical_m['T_GLU0'] = medical_m['식전혈당(공복혈당)']
medical_m['T_CREATINE'] = medical_m['혈청크레아티닌']
medical_m['T_AST'] = medical_m['혈청지오티(AST)']
medical_m['T_ALT'] = medical_m['혈청지피티(ALT)']
medical_m['T_SMOKE'] = medical_m['흡연상태'].replace({0: 1, 1: 2})
medical_m['T_DRINK'] = medical_m['음주여부'].replace({0: 1, 1: 3})

In [188]:
medical_f = medical_m[['T_SEX','T_AGE','T_HEIGHT','T_WEIGHT','T_WAIST','T_SBP','T_DBP','T_GLU0','T_CREATINE','T_AST','T_ALT','T_SMOKE','T_DRINK']]

In [189]:
medical_f

T_SEX  T_AGE  T_HEIGHT  T_WEIGHT  T_WAIST  T_SBP  T_DBP  T_GLU0  \
0          0     37       155        70     93.0  119.0   82.0    88.0   
1          0     22       165        70     79.0  152.0   91.0   114.0   
2          0     27       150        80     91.0  124.0   78.0    94.0   
3          0     37       160        60     73.0  130.0   89.0    98.0   
4          0     27       160        75     89.0  120.0   70.0    99.0   
...      ...    ...       ...       ...      ...    ...    ...     ...   
19820      0     32       155        65     75.0  122.0   71.0   103.0   
19821      0     32       160        70     86.0  129.0   83.0    80.0   
19822      0     32       160        75     88.0  137.0   89.0   123.0   
19823      0     37       155        70     89.0  112.0   74.0   103.0   
19824      0     32       155       110    109.0  133.0   95.0    92.0   

       T_CREATINE  T_AST  T_ALT  T_SMOKE  T_DRINK  
0             0.6   16.0   16.0      2.0      3.0  
1             0.7   21.0   25.0      2.0      3.0  
2             0.6   14.0   12.0      2.0      3.0  
3             0.8   22.0   25.0      2.0      1.0  
4             0.6   83.0  138.0      2.0      3.0  
...           ...    ...    ...      ...      ...  
19820         0.6   14.0    7.0      2.0      3.0  
19821         0.6   16.0   11.0      2.0      3.0  
19822         0.7   60.0  106.0      2.0      3.0  
19823         0.6   21.0   18.0      3.0      1.0  
19824         0.8   23.0   24.0      2.0      3.0  

[19825 rows x 13 columns]

In [190]:
medical_f = medical_f[['T_SEX','T_AGE','T_HEIGHT','T_WEIGHT','T_WAIST','T_SBP','T_DBP','T_SMOKE','T_DRINK']]

In [191]:
s_medical_htn = htn_scaler.transform(medical_f)

In [192]:
medical_f['PREDICT'] = htn_model.predict(s_medical_htn)

C:\Users\campus4D044\AppData\Local\Temp\ipykernel_14928\4071154099.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  medical_f['PREDICT'] = htn_model.predict(s_medical_htn)


In [194]:
medical_f['Y'] = np.where((medical_f['T_SBP'] >= 140) | (medical_f['T_DBP'] >= 90), 1, 0)

C:\Users\campus4D044\AppData\Local\Temp\ipykernel_14928\2586072766.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  medical_f['Y'] = np.where((medical_f['T_SBP'] >= 140) | (medical_f['T_DBP'] >= 90), 1, 0)


In [195]:
print(f1_score(medical_f['Y'],medical_f['PREDICT']))

0.0025244556642473968


In [193]:
medical_f.describe()

T_SEX         T_AGE      T_HEIGHT      T_WEIGHT       T_WAIST  \
count  19825.0  19825.000000  19825.000000  19825.000000  19825.000000   
mean       0.0     31.669609    159.774275     74.694578     87.472918   
std        0.0      4.670520      5.766793     13.122467     10.464950   
min        0.0     22.000000    135.000000     30.000000     57.000000   
25%        0.0     27.000000    155.000000     65.000000     80.200000   
50%        0.0     32.000000    160.000000     75.000000     86.000000   
75%        0.0     37.000000    165.000000     80.000000     93.000000   
max        0.0     37.000000    180.000000    135.000000    480.000000   

              T_SBP         T_DBP       T_SMOKE       T_DRINK       PREDICT  
count  19825.000000  19825.000000  19825.000000  19825.000000  19825.000000  
mean     126.359496     80.014678      2.105675      2.595763      0.000202  
std       12.450631      9.874303      0.307429      0.803181      0.014203  
min       72.000000     44.000000      2.000000      1.000000      0.000000  
25%      118.000000     73.000000      2.000000      3.000000      0.000000  
50%      125.000000     80.000000      2.000000      3.000000      0.000000  
75%      133.000000     85.000000      2.000000      3.000000      0.000000  
max      222.000000    150.000000      3.000000      3.000000      1.000000

In [196]:
htn_x.describe()

T_SEX        T_AGE     T_HEIGHT     T_WEIGHT      T_WAIST  \
count  8922.000000  8922.000000  8922.000000  8922.000000  8922.000000   
mean      1.660278    53.693679   160.004371    61.631361    81.464806   
std       0.473641     8.701426     8.206278     9.974941     8.767811   
min       1.000000    40.000000   117.000000    33.000000    52.000000   
25%       1.000000    47.000000   154.000000    54.000000    75.000000   
50%       2.000000    53.000000   159.000000    60.000000    81.000000   
75%       2.000000    60.000000   166.000000    68.000000    88.000000   
max       2.000000    86.000000   190.000000   122.000000   125.000000   

             T_SBP        T_DBP      T_SMOKE      T_DRINK  
count  8922.000000  8922.000000  8922.000000  8922.000000  
mean    122.906411    76.703318     1.393522     1.941157  
std      15.957519    10.399991     0.699844     0.974631  
min      72.000000    44.000000     1.000000     1.000000  
25%     110.000000    70.000000     1.000000     1.000000  
50%     120.000000    77.000000     1.000000     1.000000  
75%     132.000000    83.000000     2.000000     3.000000  
max     210.000000   134.000000     3.000000     3.000000

In [ ]:
htn_x = base_merged_r[['T_SEX','T_AGE','T_HEIGHT','T_WEIGHT','T_WAIST','T_SBP','T_DBP','T_SMOKE','T_DRINK']]
dm_x = base_merged_r[['T_GLU0','T_WEIGHT','T_WAIST','T_AGE','T_TG','T_HDL','T_DRINK','T_ALT']]
lip_x = base_merged_r[['T_TCHL','T_LDL','T_HDL','T_TG','T_WEIGHT','T_DRINK']]

htn_y = base_merged_r[['T_HTN']]
dm_y = base_merged_r[['T_DM']]
lip_y = base_merged_r[['T_LIP']]

htn_y = htn_y.replace({1.0: 0, 2.0: 1}).astype(int)
dm_y = dm_y.replace({1.0: 0, 2.0: 1}).astype(int)
lip_y = lip_y.replace({1.0: 0, 2.0: 1}).astype(int)